In [1]:
!pip install openpyxl


In [2]:
# Principales
import pandas as pd
import numpy as np

# Modificaciones 
import re
from unidecode import unidecode
import unicodedata
import datetime
import os


# visuzalizaciones
import missingno as msno
import matplotlib as plt 
import plotly.express as px 

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows',80)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)
# Ajustar la configuración para mostrar más filas 
#pd.set_option('display.max_rows', 500)

#inicializo la lista to replace
to_replace=['Null','Nan','nan','NULL',None]

# Funciones

In [4]:


# Función para normalizar texto
def normalize_text(text):
    # Reemplazar caracteres especiales y convertir a minúsculas para una comparación consistente
    text = text.replace("/", " ")  # Reemplazar la barra por espacio o eliminarla completamente
    text = text.replace("ñ", "n")  # Reemplazar la "ñ" por "n"
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    ).lower()

In [5]:
def normalizar(texto):
 return unidecode(texto).lower()

In [6]:
def actualizar_valores(df, df1, clave, columna_a_actualizar):
    """
    Actualiza los valores de una columna en un dataframe utilizando los valores de otro dataframe.
    
    :param df: DataFrame a actualizar.
    :param df1: DataFrame que contiene los valores correctos.
    :param clave: Nombre de la columna que funciona como clave primaria para hacer la correspondencia.
    :param columna_a_actualizar: Nombre de la columna que será actualizada en df.
    :return: DataFrame actualizado.
    """
    # Crear un diccionario con los valores correctos del segundo dataframe
    valores_correctos = df1.set_index(clave)[columna_a_actualizar].to_dict()
    
    # Actualizar los valores en el dataframe principal
    df[columna_a_actualizar] = df[clave].map(valores_correctos).fillna(df[columna_a_actualizar])
    
    return df


In [7]:
def actualizar_valores_conclave(df, df1, clave, columna_a_actualizar):
    """
    Actualiza los valores de una columna en un dataframe utilizando los valores de otro dataframe.
    
    :param df: DataFrame a actualizar.
    :param df1: DataFrame que contiene los valores correctos.
    :param clave: Nombre de la columna que funciona como clave primaria para hacer la correspondencia.
    :param columna_a_actualizar: Nombre de la columna que será actualizada en df.
    :return: DataFrame actualizado.
    """
    # Crear un diccionario con los valores correctos del segundo dataframe
    valores_correctos = df1.set_index(clave)[columna_a_actualizar].to_dict()
    
    # Actualizar los valores en el dataframe principal solo si coinciden las claves
    df[columna_a_actualizar] = df[clave].map(valores_correctos).combine_first(df[columna_a_actualizar])
    
    return df


In [8]:
def obtener_valores_por_columna(df, columna, valor):
    """
    Devuelve los valores de las demás columnas de la fila donde la columna especificada tiene el valor dado.
    
    :param df: DataFrame de pandas
    :param columna: Nombre de la columna para buscar el valor
    :param valor: Valor único para buscar en la columna
    :return: Diccionario con los valores de las demás columnas
    """
    # Convertir los valores de la columna a cadenas de texto y eliminar espacios en blanco
    df[columna] = df[columna].astype(str).str.strip()
    valor = str(valor).strip()
    
    fila = df.loc[df[columna] == valor]
    if not fila.empty:
        return fila.iloc[0].to_dict()
    else:
        return {}


In [9]:
def porcentaje_nulos(df, columnas):
    if isinstance(columnas, str):
        columnas = [columnas]  # Convertir a lista si es una sola columna
    resultados = {}
    for columna in columnas:
        if columna in df.columns:
            total_filas = len(df)
            nulos = df[columna].isnull().sum()
            porcentaje_nulos = (nulos / total_filas) * 100
            resultados[columna] = porcentaje_nulos
        else:
            resultados[columna] = 'Column not found'
    return resultados



In [10]:
def obtener_valores_por_columnaold(df, columna, valor):
    """
    Devuelve los valores de las demás columnas de la fila donde la columna especificada tiene el valor dado.
    
    :param df: DataFrame de pandas
    :param columna: Nombre de la columna para buscar el valor
    :param valor: Valor único para buscar en la columna
    :return: Diccionario con los valores de las demás columnas
    """
    # Eliminar espacios en blanco de los valores en la columna y del valor de búsqueda
    df[columna] = df[columna].str.strip()
    valor = valor.strip()
    
    fila = df.loc[df[columna] == valor]
    if not fila.empty:
        return fila.iloc[0].to_dict()
    else:
        return {}


# Carga de datos

In [11]:

print(os.getcwd())

c:\Users\extas\Documents\GitHub\precios-de-vivienda-madrid\codigo\01-preprocesamiento-de-datos\Revision_columnas


In [12]:
df17= pd.read_csv('./data/dfpisosCompleto17012025.csv',index_col=False)

C:\Users\extas\AppData\Local\Temp\ipykernel_21052\2425994981.py:1: DtypeWarning: Columns (3,5,14,26,30,68,69,74,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df17= pd.read_csv('./data/dfpisosCompleto17012025.csv',index_col=False)


In [13]:
dfbarrios= pd.read_excel('../02-datos-limpios/distr_barrios_codpostal.xlsx')

In [14]:
dfadicionales= pd.read_csv('../02-datos-limpios/Datos_adicionales.csv',sep=";", index_col=False)

In [15]:
dfbarrios.head(2)

,cod_distrito,distrito,cod_barrio,cod_barrio_2,barrio,codigo_Postal
0,1,Centro,11,1,Palacio,28013
1,1,Centro,12,2,Embajadores,28012


In [16]:
df17 = df17.loc[:, ~df17.columns.str.contains('^Unnamed')]

In [17]:
df17.shape

(10430, 110)

In [18]:
df17.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

# **_Revision primeras 31 columnas Gilberto_**  

Codigo trasladado

In [19]:
# Selecciono las primeras 31 columnas. De la 0 a la 30.
df_131=df17.iloc[:, :31]

In [ ]:
# Checo la forma de mi dataframe y que realmente tome 31 columnas.
df_131.shape
# Muestreo de las primeras 5 filas 
df_131.head()
#Muestreo de las ultimas 5 filas 
df_131.tail()
# Muestreo de 5 filas aleatorias 
df_131.sample(5)


In [21]:
# Chqueo de nulos en todas las columnas

df_131.isna().sum()
# Muestreo estadistico de las columnas

title                     0
url                       0
precio                    0
precio_anterior       10376
descuento             10376
EUR/m2                 2379
m2_constr                28
distrito                 22
cod_distrito             69
barrio                  411
cod_barrio              386
zona                   4805
calle                  9368
detalle                4506
consumoce_ano          7089
letrace                9039
emisiones_co2          6828
tipologia                 0
propiedad_completa    10319
estado                 4761
amueblado              7533
planta                 4645
habitaciones           7183
dormitorios             191
banos                   305
balcon                 4483
terraza                4359
garaje                 8374
trastero               1700
ascensor               2311
porteria              10278
dtype: int64

In [22]:

df_131.describe().T

,count,mean,std,min,25%,50%,75%,max
precio_anterior,54.00,812929.63,977732.38,98100.00,197750.00,432000.00,949875.00,4900000.00
EUR/m2,8051.00,3628.73,4001.52,0.00,6.58,2846.00,6246.50,27708.00
m2_constr,10402.00,248.27,6465.93,0.00,67.00,97.00,150.00,656600.00
cod_distrito,10361.00,7.71,5.75,0.00,4.00,6.00,12.00,21.00
cod_barrio,10044.00,81.49,57.31,1.00,41.00,65.00,124.00,215.00
propiedad_completa,111.00,1.00,0.00,1.00,1.00,1.00,1.00,1.00
dormitorios,10239.00,2.81,1.43,0.00,2.00,3.00,3.00,24.00
terraza,6071.00,0.32,0.47,0.00,0.00,0.00,1.00,1.00
trastero,8730.00,0.16,0.36,0.00,0.00,0.00,0.00,1.00


In [23]:
df_131.describe().T
# Muestreo de tipos de columnas y nulos 
df_131.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10430 entries, 0 to 10429
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               10430 non-null  object 
 1   url                 10430 non-null  object 
 2   precio              10430 non-null  object 
 3   precio_anterior     54 non-null     float64
 4   descuento           54 non-null     object 
 5   EUR/m2              8051 non-null   float64
 6   m2_constr           10402 non-null  float64
 7   distrito            10408 non-null  object 
 8   cod_distrito        10361 non-null  float64
 9   barrio              10019 non-null  object 
 10  cod_barrio          10044 non-null  float64
 11  zona                5625 non-null   object 
 12  calle               1062 non-null   object 
 13  detalle             5924 non-null   object 
 14  consumoce_ano       3341 non-null   object 
 15  letrace             1391 non-null   object 
 16  emis

In [24]:
# Chequeo de nulos en la columna title 
df_131["title"].isna().sum()
# Filtro para checar si las filas que no tienen valores en title tienen valores en otras columnas importantes
#filas_con_nulos= df_131[df_131["title"].isnull()]
#ilas_con_nulos

0

In [25]:
df_131["title"].head(20)
df_131["title"].sample(20)

8445                   Piso en venta en CALLE ARROYO FONTARRON Fontarrón Moratalaz Madrid Madrid
2264                                    Piso de tres habitaciones Cartagena, Prosperidad, Madrid
6099                        Piso en venta en Calle de Ana Teresa, 1, cerca de Paseo de la Ermita
3900                                                         Piso en venta en Calle del Pelícano
4314                                         Piso en venta en Calle de Teresita González Quevedo
2188                                          Piso de dos habitaciones 50 m², Marroquina, Madrid
9307                                      Piso de cuatro habitaciones 182 m², Niño Jesús, Madrid
4442                                                                        Piso en venta en Sol
7296                                             Piso en venta en Avenida de las Fuerzas Armadas
4104                                                        Piso en venta en Calle de Durazno, 9
6471                          

In [26]:
df_131["url"].sample(10)
df_131["url"].isna().sum()

0

In [27]:
# Filtro de filas con valores pequeños que no podrian ser el coste de un inmueble
df_131[df_131["precio"] == 1]

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria
758,"Piso de una habitación planta baja, Sol, Madrid",https://www.indomio.es/anuncios/95899949/,1.00,NaN,NaN,NaN,NaN,Centro,1.00,Sol,16.00,Sol,NaN,Piso de una habitación planta baja,desconocido,NaN,No indicado,Piso,NaN,NaN,NaN,Planta baja,1,1.00,1,0.00,NaN,NaN,1.00,1.00,NaN


se revisa la url https://www.indomio.es/anuncios/95899949/ y en l web aparece con valor 1€, lo consideramos un error, elimino la linea del dataset

In [28]:
df17.shape

(10430, 110)

In [29]:
# Eliminar filas donde la columna 'precio' es igual a 1
df17 = df17[df17["precio"] != 1]


In [30]:
df17.shape

(10429, 110)

In [31]:

# Filtro de las filas con valores en precio anteior 
filas_sin_nulos= df_131[~df_131["precio_anterior"].isnull()]
filas_sin_nulos.head(4)
# Aqui filtre los links de los inmuebles con el precio anterior para checar algunos directamente y 
# ver si habia algo relevante del porque del cambio del precio.
filas_sin_nulos["url"]
# Chequeo de porcentaje de nulos en la columna precio_anterior
df_131["precio_anterior"].isna().mean()*100
df_131.shape
# Porcentaje de nulos 
df_131["descuento"].isna().mean()*100
#Porcentaje de nulos
df_131["EUR/m2"].isna().mean()*100
# Chequeo de nulos totales
df_131["EUR/m2"].isna().sum()
df_131["EUR/m2"].describe()
# Filtro de valores iguales a 0
filas_valor_eur= df_131[df_131["EUR/m2"] == 0]
filas_valor_eur

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria
1029,"Piso 656600 m², Recoletos, Madrid",https://www.indomio.es/anuncios/96246263/,124150.00,NaN,NaN,0.00,656600.00,Salamanca,4.00,Recoletos,41.00,Recoletos,NaN,Piso 656600 m²,desconocido,G,No indicado,Piso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN


In [32]:
# Filtro de valores 
filas_valor_eur_pequeños= df_131[df_131["EUR/m2"] <=100 ]
filas_valor_eur_pequeños.tail(5)
# Filtro para valores menores a 99 con un decimal. de esa forma los valores podran ser filtrados para no tener un precio mayor a 4 cifras por metro cuadrado
# que es lo que se ha notado donde predominan los valores.  Aqui cheque los valores con Data Wrangler y pude visualizar que los valores no son correctos. A algunos 
# les sirve solo colocarle 000 ceros extras al valor por metro cuadrado pero a otros no les sirve eso. 

filtered_df_131 = df_131[df_131['EUR/m2'].apply(lambda x: x < 99 and (x * 10).is_integer())]
filtered_df_131.head(5)
# Filtro por valores que parecen atipicos e irregulares para identificar vericidad
filas_valor_eur2= df_131[df_131["EUR/m2"] >= 20000]
filas_valor_eur2.head()
# Filtro del link por valores que parecen atipicos e irregulares para identificar vericidad
filas_valor_eur2= df_131[df_131["EUR/m2"] >= 20000]
filas_valor_eur2["url"]
# Porcentaje de nulos por columna
df_131.isna().mean()*100

title                 0.00
url                   0.00
precio                0.00
precio_anterior      99.48
descuento            99.48
EUR/m2               22.81
m2_constr             0.27
distrito              0.21
cod_distrito          0.66
barrio                3.94
cod_barrio            3.70
zona                 46.07
calle                89.82
detalle              43.20
consumoce_ano        67.97
letrace              86.66
emisiones_co2        65.47
tipologia             0.00
propiedad_completa   98.94
estado               45.65
amueblado            72.22
planta               44.53
habitaciones         68.87
dormitorios           1.83
banos                 2.92
balcon               42.98
terraza              41.79
garaje               80.29
trastero             16.30
ascensor             22.16
porteria             98.54
dtype: float64

## m2_constr

In [33]:
# Analisis descriptivo de la columna m2/constr 
df17["m2_constr"].describe()

count    10402.00
mean       248.27
std       6465.93
min          0.00
25%         67.00
50%         97.00
75%        150.00
max     656600.00
Name: m2_constr, dtype: float64

In [34]:

#Filtro de valores menores a 20 metros cuadrados. Se analizo con 10 tambien y siguen siendo mas de 200
df17[df17["m2_constr"] <= 10]['web'].unique()

array(['pisos.com', 'pisosmadrid'], dtype=object)

In [35]:
df17.query("m2_constr<= 10 & web=='pisosmadrid'")

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,€_comunidad_mes,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,m2_utiles,vista_zona,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,orientacion_este,orientacion_norte,"orientacion_norte,_sur",orientacion_oeste,orientacion_sur,"orientacion_sur,_este",sistema_alarma,n.visitas,grupo_cocina
9660,Piso en venta en Malasana Madrid,https://www.pisosmadrid.com.es/propiedad/3912876-piso-en-venta-en-malasana-madrid,550000,NaN,NaN,NaN,0.00,Centro,1.00,Sol,16.00,Malasana,NaN,NaN,NaN,NaN,NaN,Piso,NaN,NaN,0.0,NaN,NaN,3.00,2,NaN,NaN,0.0,0.00,NaN,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,AVC INMOBILIARIO VENDE PISO Este magnifico piso abuhardillado esta ubicado en el efervescente Barrio de Malasana a tan solo unos pasos de la Gran Via Rodeado de boutiques restaurantes modernos y teatros se encuentra,NaN,NaN,NaN,pisosmadrid,24/10/2024,NaN,0.00,1.00,NaN,NaN,3912876,90.00,2024-06-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,28013.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.00,sin_informacion
10172,Piso en venta en Pueblo Nuevo Madrid,https://www.pisosmadrid.com.es/propiedad/4056165-piso-en-venta-en-pueblo-nuevo-madrid,212500,NaN,NaN,NaN,0.00,Ciudad Lineal,15.00,PuebloNuevo,152.00,Pueblo Nuevo,NaN,NaN,NaN,NaN,NaN,Piso,NaN,NaN,0.0,NaN,NaN,2.00,1,NaN,NaN,0.0,0.00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,Piso 2 dormitorios 1 banos 0 garajes Para reformar en Madrid Madrid Ubicado en el barrio de Pueblo Nuevo Madrid Piso exterior que consta de 2 habitaciones 1 bano saloncomedor cocina y terraza La vivienda esta,NaN,NaN,NaN,pisosmadrid,24/10/2024,NaN,0.00,1.00,NaN,NaN,4056165,10.00,2024-10-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,28017.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.00,sin_informacion
10176,Apartamento en venta en Centro Madrid,https://www.pisosmadrid.com.es/propiedad/4056174-apartamento-en-venta-en-centro-madrid,250000,NaN,NaN,NaN,0.00,Carabanchel,11.00,Opañel,112.00,Centro,NaN,NaN,NaN,NaN,NaN,Apartamento,NaN,NaN,0.0,NaN,NaN,3.00,1,NaN,NaN,0.0,0.00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,Apartamento 3 dormitorios 1 banos 0 garajes Para reformar en Madrid Madrid Ubicado en el barrio de Opanel Carabanchel Piso de 100 m2 construidos distribuido en saloncomedor cocina 3 habitaciones y 2 banos La,NaN,NaN,NaN,pisosmadrid,24/10/2024,NaN,0.00,1.00,NaN,NaN,405

# En el caso de pisos madrid se visita la web, no existe campo con el valor pero lo poner en la descripcion, se asigna el valor de 2 de los  pisos

In [36]:
# Asignar valor específico si la URL coincide
df17.loc[df17["url"] == "https://www.pisosmadrid.com.es/propiedad/3912876-piso-en-venta-en-malasana-madrid", "m2_constr"] = 131

df17.loc[df17["url"] == "https://www.pisosmadrid.com.es/propiedad/4056174-apartamento-en-venta-en-centro-madrid", "m2_constr"] = 100



In [37]:
df17.query("m2_constr<= 10 & web=='pisos.com'")

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,€_comunidad_mes,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,m2_utiles,vista_zona,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,orientacion_este,orientacion_norte,"orientacion_norte,_sur",orientacion_oeste,orientacion_sur,"orientacion_sur,_este",sistema_alarma,n.visitas,grupo_cocina
3111,"Apartamento en venta en Calle de los Misterios, 16, cerca de Calle de la Virgen de Lluc",https://www.pisos.com/comprar/apartamento-ciudad_lineal_quintana-48353042971_100500/,220000.00,NaN,NaN,6875.00,2.00,Ciudad Lineal,15.00,Quintana,153.00,NaN,NaN,"Si estás buscando una vivienda para independizarte o para invertir , ¡Deja de buscar, esta es tu oportunidad!\n\nAsr home te presenta genial apartamento, con garaje y trastero en una de las mejores zonas de madrid. \n\nEste piso es especial porque: por la luz que tiene, la ubicación y por tener todo lo que necesitas.\n\nDistribucion: al ser un apartamento , todo esta a la mano, la cocina y salón todo junto y el baño independiente.\n\nEquipamiento y calidades: el piso se encuentra para entrar a vivir con calidades de construcción muy buenas y en condiciones muy aceptables.\n\nComunicaciones: se ubica cerca del metro ciudad lineal y múltiples paradas de autobuses que le llevaran a cualquier lugar del centro de madrid. \n\nComercio, servicios, ocio: en esta zona hay gran variedad de comercio, restauración, centros médicos y hospitales, \n\nCentros educativos: en el sector de la enseñanza la zona cuenta con una estupendas guarderías y escuela de primaria. Para los otros niveles hay colegios públicos y privados.\n\nIbi 414.39 €/año\nComunidad 114,17 €/mes\n\nTe financiamos hasta el 100% no dudes en consultarno.",NaN,NaN,NaN,Apartamento,NaN,NaN,NaN,6875.0,NaN,1.00,32.0,0.00,0.00,NaN,1.00,1.00,NaN,Gas natural,Entre 10 y 20 años,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pisos.com,2024-10-19,Disponible,NaN,NaN,NaN,153.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sin_informacion
3141,"Apartamento en venta en Calle del Teniente Coronel Noreña, 22",https://www.pisos.com/comprar/apartamento-legazpi_barrio28045-29263829710_109700/,410415.00,NaN,NaN,6728.00,1.00,Arganzuela,2.00,Legazpi,24.00,NaN,NaN,Apartamento en un edificio de obra nueva de 61 m2 de superficie construida y ubicado en la zona de Madrid Capital. Está distribuido en 1 habitación.,NaN,NaN,NaN,Apartamento,NaN,A estrenar,NaN,61.0,NaN,1.00,1.0,0.00,0.00,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

En pisos.com hay 198 filas con m2_contr erronea, se va al origen para ver que transformación se realizó

In [38]:
#utilizo el fichero pisoscom_limpio que parece que aun tenia los valores de m2_constr
dfpcomlimpio= pd.read_csv('../02-datos-limpios/pisoscom_limpio.csv',index_col=False)

In [39]:
dfpcomlimpio.columns

Index(['title', 'ubicacion', 'precio', 'habitaciones', 'banos', 'm2_constr',
       'planta', 'EUR/m2', 'estado', 'url', 'fecha_descarga', 'web',
       'distrito', 'barrio', 'cod_distrito', 'cod_barrio', 'tipologia'],
      dtype='object')

In [40]:
dfpcomlimpio.query("url=='https://www.pisos.com/comprar/piso-villaverde_san_andres28021-4918181785_109700/'")[["m2_constr"]]

,m2_constr
910,88.00


In [41]:
df17.m2_constr.info()

<class 'pandas.core.series.Series'>
Index: 10429 entries, 0 to 10429
Series name: m2_constr
Non-Null Count  Dtype  
--------------  -----  
10402 non-null  float64
dtypes: float64(1)
memory usage: 163.0 KB


In [42]:
#convierto a float m2_constr en el dataframe de pisos
# Convertir la columna 'nombre_columna' a tipo float
dfpcomlimpio['m2_constr'] = dfpcomlimpio['m2_constr'].astype(float)


In [43]:
#ahora ponemos el valor que tenemos en dfpcomlimpio cuando coincide la web
df17=actualizar_valores_conclave(df17, dfpcomlimpio, 'url', 'm2_constr')

In [44]:
df17.query("m2_constr<= 10 & web=='pisos.com'")['url']

3319                     https://www.pisos.com/comprar/atico-latina_aluche28047-4826295847_109700/
3796    https://www.pisos.com/comprar/estudio-centro_embajadores_lavapies28012-42547139321_101800/
5873                      https://www.pisos.com/comprar/piso-latina_aluche28047-4505819831_109700/
5874                      https://www.pisos.com/comprar/piso-latina_aluche28047-4568954085_109700/
5888                      https://www.pisos.com/comprar/piso-latina_aluche28047-4696641924_109700/
5889                      https://www.pisos.com/comprar/piso-latina_aluche28047-4715872594_109700/
Name: url, dtype: object

#en la web ponen que tiene 10 m2, y por las fotos lo parece

In [45]:
df17["m2_constr"].value_counts(dropna=False)

m2_constr
70.00       183
60.00       172
90.00       147
80.00       140
75.00       137
           ... 
1554.00       1
694.00        1
974.00        1
654.00        1
12054.00      1
Name: count, Length: 561, dtype: int64

In [46]:
df17.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

In [47]:
#voy a mirar los valores de m2_constr>1000
df17.query("m2_constr>= 5000")[['url','dormitorios','m2_constr']]

,url,dormitorios,m2_constr
1029,https://www.indomio.es/anuncios/96246263/,NaN,656600.00
9607,https://www.pisosmadrid.com.es/propiedad/3891215-piso-en-venta-en-madrid,1.00,5275.00
9608,https://www.pisosmadrid.com.es/propiedad/3891216-piso-en-venta-en-madrid,1.00,5275.00
9969,https://www.pisosmadrid.com.es/propiedad/4018493-piso-en-venta-en-conde-orgaz-madrid,2.00,7995.00
10130,https://www.pisosmadrid.com.es/propiedad/4049211-piso-en-venta-en-simancas-madrid,3.00,10391.00
10157,https://www.pisosmadrid.com.es/propiedad/4054068-piso-en-venta-en-canillejas-madrid,2.00,5641.00
10159,https://www.pisosmadrid.com.es/propiedad/4055005-piso-en-venta-en-ciudad-lineal-madrid,2.00,5641.00
10199,https://www.pisosmadrid.com.es/propiedad/4058113-apartamento-en-venta-en-ciudad-lineal-madrid,2.00,5641.00
10362,https://www.pisosmadrid.com.es/propiedad/4073872-piso-en-venta-en-ensanche-de-vallecas-madrid,2.00,8844.00
10402,https://www.pisosmadrid.com.es/propiedad/4084004-atico-en-venta-en-san-isidro-madrid,1.00,10554.00


Se detecta un error en los datos de m2_constr de pisos madrid, al tener algunos valores los decimales con puntos, cargo el original de pisos madrid para arreglar este valor

In [48]:
#como en el origen está bien, traigo los valores de pisos madrid
dfpmlimpio= pd.read_csv('../02-datos-limpios/dfpisosmadrid_paraunir.csv',index_col=False)

In [49]:
dfpmlimpio['m2_constr'] = dfpmlimpio['m2_constr'].astype(float)

In [50]:
#ahora ponemos el valor que tenemos en dfpcomlimpio cuando coincide la web
df17=actualizar_valores_conclave(df17, dfpmlimpio, 'url', 'm2_constr')

In [51]:
df17.query("m2_constr>= 5000")[['url','dormitorios','m2_constr']]

,url,dormitorios,m2_constr
1029,https://www.indomio.es/anuncios/96246263/,NaN,656600.00


En el scraping aparece con ese valor, pero entiendo que es un error

In [52]:
# Eliminar la fila donde la columna 'url' es igual a la URL especificada
df17 = df17[df17["url"] != "https://www.indomio.es/anuncios/96246263/"]


In [53]:
# Resetear el índice del DataFrame
df17.reset_index(drop=True, inplace=True)

In [54]:
# Poner en mayúsculas la primera letra de cada palabra en la columna 'distrito'
df17['distrito'] = df17['distrito'].str.title()


In [55]:
df17["distrito"].unique()

array([' Salamanca', ' Barajas', ' Arganzuela', ' Chamartín', ' Tetuán',
       ' Chamberí', 'Villaverde', ' Latina', ' Centro',
       ' Fuencarral-El Pardo', ' Villa De Vallecas',
       ' Puente De Vallecas', ' Retiro', ' San Blas-Canillejas',
       ' Ciudad Lineal', 'Hortaleza', ' Moncloa-Aravaca', ' Carabanchel',
       'Carabanchel', ' Hortaleza', ' Vicálvaro', ' Usera', 'Salamanca',
       'Puente De Vallecas', 'Ciudad Lineal', 'Fuencarral-El Pardo',
       'Barajas', ' Moratalaz', 'Tetuan', ' Villaverde',
       'Villa De Vallecas', 'Usera', 'Moratalaz', 'Chamberi', 'Centro',
       'Latina', 'San Blas-Canillejas', 'Vicálvaro', nan, 'Retiro',
       'Vicalvaro', 'Chamartin', 'Arganzuela', 'Moncloa-Aravaca'],
      dtype=object)

In [56]:


# Ajustar la configuración para mostrar más caracteres en las columnas
pd.set_option('display.max_colwidth', None)




In [57]:
df17.query("distrito.isnull()")[['distrito','url','cod_distrito','barrio','descripcion_extendida']]

,distrito,url,cod_distrito,barrio,descripcion_extendida
3329,NaN,https://www.pisos.com/comprar/atico-los_berrocales28031-4666816647_109700/,19.00,NaN,NaN
6038,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-3500496548_109700/,19.00,NaN,NaN
6039,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4335511605_109700/,19.00,NaN,NaN
6040,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4353985260_109700/,19.00,NaN,NaN
6041,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4355464537_109700/,19.00,NaN,NaN
6042,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4396532976_109700/,19.00,NaN,NaN
6043,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4418412482_109700/,19.00,NaN,NaN
6044,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4425541774_109700/,19.00,NaN,NaN
6045,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4510709962_109700/,19.00,NaN,NaN
6046,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4568166771_109700/,19.00,NaN,NaN


In [58]:
# Asignar valor específico si la URL coincide

df17.loc[df17["url"] == "https://www.pisosmadrid.com.es/propiedad/3895829-piso-en-venta-en-madrid", "cod_barrio"] = 133
df17.loc[df17["url"] == "https://www.pisosmadrid.com.es/propiedad/3906133-piso-en-venta-en-madrid", "cod_barrio"] = 65



No tenemos forma de conocer la ubicacion de https://www.pisosmadrid.com.es/propiedad/3930386-piso-en-venta-en-madrid	, lo elimino del dataset

In [59]:
# Eliminar la fila donde la columna 'url' es igual a la URL especificada
df17 = df17[df17["url"] != "https://www.pisosmadrid.com.es/propiedad/3930386-piso-en-venta-en-madrid"]

# Resetear el índice del DataFrame
df17.reset_index(drop=True, inplace=True)


In [60]:
#Con el codigo de barrio

In [61]:
df17.query("cod_barrio.notnull() & cod_distrito.isnull()")[['distrito','cod_distrito','barrio','cod_barrio','descripcion_extendida']]

,distrito,cod_distrito,barrio,cod_barrio,descripcion_extendida
9618,NaN,NaN,NaN,133.00,PISO EN MADRIDVivienda situada en la Calle Mariano Benlliure en Madrid VIVIENDA CON INCIDENCIAS INMUEBLE SIN POSESION El estado de ocupado yo los impedimentos para que el inmueble pueda ser visitado libremente p
9639,NaN,NaN,NaN,65.00,BRLASVICTORIASbr \nADESCUBRE LAS VICTORIASbr \nBE REAL presenta LAS VICTORIASa PromociA3n de Obra Nueva bajo venta sobre plano en Madrid situada en la Calle Moquetas 11 Este exclusivo proyecto cuenta con 10


In [62]:
# Filtrar filas donde 'cod_barrio' no es nulo y 'cod_distrito' es nulo
filas = df17.query("cod_barrio.notnull() & cod_distrito.isnull()")

# Iterar sobre las filas filtradas
for idx, fila in filas.iterrows():
    bar = fila['cod_barrio']
    valores = obtener_valores_por_columna(dfbarrios, 'cod_barrio', bar)
    
    if valores:
        if pd.isna(df17.at[idx, 'barrio']): 
            df17.at[idx, 'barrio'] = valores.get('barrio')       
        if pd.isna(df17.at[idx, 'cod_barrio']): 
            df17.at[idx, 'cod_barrio'] = valores.get('cod_barrio')
        if pd.isna(df17.at[idx, 'cod_distrito']): 
            df17.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
        if pd.isna(df17.at[idx, 'distrito']): 
            df17.at[idx, 'distrito'] = valores.get('distrito')
        df17.at[idx, 'zona'] = 'bar'



In [63]:
df17.query("cod_barrio.isnull() & cod_distrito.isnull()")[['distrito','cod_distrito','barrio','cod_barrio','descripcion_extendida']]

,distrito,cod_distrito,barrio,cod_barrio,descripcion_extendida
9562,Ciudad Lineal,NaN,NaN,NaN,DP2020 vende piso en edificio residencial con ascensor ubicado en la localidad de MadridConsta de una superficie de 102m2 bien distribuida 3 dormitorios amplios 2 banos un amplio y luminoso saloncomedor con salida a
9592,Chamberi,NaN,NaN,NaN,Vivienda situada en Calle Diario de la Nacion en Madrid VIVIENDA CON INCIDENCIAS INMUEBLE SIN POSESION OKUPADOEl estado de ocupado yo los impedimentos para que el inmueble pueda ser visitado libremente por los
9608,Ciudad Lineal,NaN,NaN,NaN,LF INMOGROUP Torrejon vende en exclusiva este fabuloso piso en una de las mejores zonas de Madrid Consta de 4 dormitorios y dos banos de marmol completos Salon independiente cocina amueblada y equipada con horno pl
9625,Vicalvaro,NaN,NaN,NaN,PISO EN MADRIDVivienda situada en la Calle Abardero en la localidad de Madrid VIVIENDA CON INCIDENCIAS INMUEBLE SIN POSESION El estado de ocupado yo los impedimentos para que el inmueble pueda ser visitado libre
9641,Chamartin,NaN,NaN,NaN,ADESCUBRE LAS VICTORIASbr \nBE REAL presenta LAS VICTORIASa PromociA3n de Obra Nueva bajo venta sobre plano en Madrid situada en la Calle Moquetas 11 Este exclusivo proyecto cuenta con 10 impresionantes viviend
9642,Vicalvaro,NaN,NaN,NaN,CSJ INVERSIONES VENDE en exclusiva Fantastico piso de una superficie construida de 60m2 2a Planta Ascensor con llave\nLa vivienda se distribuye en salon comedor independiente 1dormitorio
9644,Chamartin,NaN,NaN,NaN,CSJ presenta este exclusivo piso ubicado en un edificio principal de siete plantas y un edificio anexo de dos plantas de altura El edificio ha sido rehabilitado en su totalidad manteniendo
9652,Ciudad Lineal,NaN,NaN,NaN,Piso en excelente estado para entrar a vivir\n\nCon tres dormitorios y dos baAos Es una VPO enA RAcgimen Especial Se puede comprar con requisitos del IVIMA aDarse de alta como demandante de vivienda no ganar mAs
9654,Fuencarral-El Pardo,NaN,NaN,NaN,GUERRA Y MENDEZ Propiedades ofrece en venta chalet adosado en pequena urbanizacion con piscina comunitaria Se distribuye en cuatro plantas con cinco dormitorios y cuatro cuartos de bano uno de ellos en suite a
9659,Chamartin,NaN,NaN,NaN,ESTUDIO HOME MADRID OFRECE atico de 103 m2 construidos de vivienda y tiene como anejo inseparable el uso y disfrute de la terraza que da acceso de 72m2 EL INMUEBLE ESTA ACTUALMENTE ALQUILADO HASTA JULIO DE 2025 CON U


In [64]:
dfbarrios

,cod_distrito,distrito,cod_barrio,cod_barrio_2,barrio,codigo_Postal
0,1,Centro,11,1,Palacio,28013
1,1,Centro,12,2,Embajadores,28012
2,1,Centro,13,3,Cortes,28014
3,1,Centro,14,4,Justicia,28004
4,1,Centro,15,5,Universidad,28015
...,...,...,...,...,...,...
126,20,San Blas-Canillejas,208,8,Salvador,28037
127,21,Barajas,211,1,Alameda de Osuna,28042
128,21,Barajas,212,2,Aeropuerto,28042
129,21,Barajas,213,3,Corralejos,28042


In [65]:
#creo una columna en el dataframe dfbarrios para normalizar los distritos


# Eliminar acentos de la columna 'distrito'
dfbarrios['distrito_n'] = dfbarrios['distrito'].apply(lambda x: unidecode(x))
dfbarrios['barrio_n'] = dfbarrios['barrio'].apply(lambda x: unidecode(x))
dfbarrios['barrio_nn']=dfbarrios['barrio'].apply(lambda x: normalizar(x))


In [66]:
dfbarrios.query("distrito==' Vicálvaro'")

,cod_distrito,distrito,cod_barrio,cod_barrio_2,barrio,codigo_Postal,distrito_n,barrio_n,barrio_nn
115,19,Vicálvaro,191,1,Casco Histórico de Vicálvaro,28032,Vicalvaro,Casco Historico de Vicalvaro,casco historico de vicalvaro
116,19,Vicálvaro,192,2,Valdebernardo,28032,Vicalvaro,Valdebernardo,valdebernardo
117,19,Vicálvaro,193,3,Valderrivas,28032,Vicalvaro,Valderrivas,valderrivas
118,19,Vicálvaro,194,4,El Cañaveral,28032,Vicalvaro,El Canaveral,el canaveral


In [67]:
# Reemplazar la palabra "De" por "de" en la columna 'distrito'
df17['distrito'] = df17['distrito'].str.replace(r'\bDe\b', 'de', regex=True)


In [68]:
# Filtrar filas donde 'cod_barrio' no es nulo y 'cod_distrito' es nulo
filas = df17.query("(cod_barrio.isnull() & cod_distrito.isnull()) | (cod_barrio.isnull() & cod_distrito == 0)")
#ahora tengo nombre distrito pero no codigo
# Iterar sobre las filas filtradas
for idx, fila in filas.iterrows():
   
    bar= unidecode(fila['distrito'])
   

    valores = obtener_valores_por_columna(dfbarrios, 'distrito_n', bar)
    
    if valores:
        #if pd.isna(df17.at[idx, 'barrio']): 
         #   df17.at[idx, 'barrio'] = valores.get('barrio')       
        #if pd.isna(df17.at[idx, 'cod_barrio']): 
         #   df17.at[idx, 'cod_barrio'] = valores.get('cod_barrio')
        if pd.isna(df17.at[idx, 'cod_distrito']): 
            df17.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
        if pd.isna(df17.at[idx, 'distrito']): 
            df17.at[idx, 'distrito'] = valores.get('distrito')
        #df17.at[idx, 'zona'] = 'bar'

In [69]:


# Filtrar filas donde 'cod_barrio' no es nulo y 'cod_distrito' es nulo
filas = df17.query("cod_barrio.notnull() ")

# Iterar sobre las filas filtradas
for idx, fila in filas.iterrows():
    if pd.notna(fila['zona']):
        bar = unidecode(fila['zona'])
        #print (bar)
        valores = obtener_valores_por_columna(dfbarrios, 'distrito_n', bar)
        
        if valores:           
            #if pd.isna(df17.at[idx, 'cod_distrito']): 
                n= valores.get('cod_distrito')
                df17.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
            #if pd.isna(df17.at[idx, 'distrito']): 
                df17.at[idx, 'distrito'] = valores.get('distrito')
           


In [70]:
# Eliminar espacios en blanco de la columna 'zona'
df17['zona'] = df17['zona'].str.strip()

In [71]:
# Eliminar todos los espacios en blanco de la columna 'zona'
df17['zona'] = df17['zona'].str.replace(' ', '')

In [72]:
dfbarrios.tail(50)

,cod_distrito,distrito,cod_barrio,cod_barrio_2,barrio,codigo_Postal,distrito_n,barrio_n,barrio_nn
81,13,Puente de Vallecas,133,3,Palomeras Bajas,28018,Puente de Vallecas,Palomeras Bajas,palomeras bajas
82,13,Puente de Vallecas,134,4,Palomeras Sureste,28018,Puente de Vallecas,Palomeras Sureste,palomeras sureste
83,13,Puente de Vallecas,135,5,Portazgo,28018,Puente de Vallecas,Portazgo,portazgo
84,13,Puente de Vallecas,136,6,Numancia,28038,Puente de Vallecas,Numancia,numancia
85,14,Moratalaz,141,1,Pavones,28030,Moratalaz,Pavones,pavones
86,14,Moratalaz,142,2,Horcajo,28030,Moratalaz,Horcajo,horcajo
87,14,Moratalaz,143,3,Marroquina,28030,Moratalaz,Marroquina,marroquina
88,14,Moratalaz,144,4,Media Legua,28030,Moratalaz,Media Legua,media legua
89,14,Moratalaz,145,5,Fontarrón,28030,Moratalaz,Fontarron,fontarron
90,14,Moratalaz,146,6,Vinateros,28030,Moratalaz,Vinateros,vinateros


In [73]:
# Filtrar filas donde 'cod_barrio' no es nulo y 'cod_distrito' es nulo
filas = df17.query("cod_barrio.isnull() ")

# Iterar sobre las filas filtradas
for idx, fila in filas.iterrows():
    if pd.notna(fila['zona']):
        bar = unidecode(fila['zona'])
        if bar=='villaverdealto,cascohistoricodevillaverde':           
            bar='villaverde alto'
        if bar=='loscarmenes':
            bar='los carmenes'
        if bar=='angeles':
            bar='los angeles'
        if bar=='cascohistoricodevicalvaro':
            bar='casco historico de vicalvaro'
        if bar=='elsalvador':
            bar='salvador'
        if bar=='Malasana':
            bar='universidad'
        if bar=='Ventilla':
            bar='almenara'
        if bar=='Orense':
            bar='castillejos'
        if bar=='VillaverdeBajo':
            bar='los rosales'
        if bar=='LaLatina':
            bar='palacio'
       
       #print (fila['zona'])
        valores = obtener_valores_por_columna(dfbarrios, 'barrio_nn', bar)
        print(bar)    
        if valores:  
           # if bar=='valdefuentes':
            print(bar)         
            #if pd.isna(df17.at[idx, 'cod_distrito']): 
            n= valores.get('cod_distrito')
            df17.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
            #if pd.isna(df17.at[idx, 'distrito']): 
            df17.at[idx, 'distrito'] = valores.get('distrito')
            #if pd.isna(df17.at[idx, 'barrio']): 
            df17.at[idx, 'barrio'] = valores.get('barrio')       
            #if pd.isna(df17.at[idx, 'cod_barrio']): 
            df17.at[idx, 'cod_barrio'] = valores.get('cod_barrio')

villaverde alto
villaverde alto
valdefuentes
valdefuentes
valdefuentes
valdefuentes
villaverde alto
villaverde alto
valverde
valverde
los carmenes
los carmenes
los carmenes
los carmenes
valdefuentes
valdefuentes
villaverde alto
villaverde alto
valdefuentes
valdefuentes
los carmenes
los carmenes
mirasierra
mirasierra
los angeles
los angeles
valdefuentes
valdefuentes
villaverde alto
villaverde alto
casco historico de vicalvaro
casco historico de vicalvaro
los carmenes
los carmenes
valdefuentes
valdefuentes
villaverde alto
villaverde alto
valdefuentes
valdefuentes
villaverde alto
villaverde alto
valdefuentes
valdefuentes
valdefuentes
valdefuentes
valdefuentes
valdefuentes
salvador
salvador
valdefuentes
valdefuentes
valdefuentes
valdefuentes
los carmenes
los carmenes
villaverde alto
villaverde alto
valdefuentes
valdefuentes
valdefuentes
valdefuentes
salvador
salvador
villaverde alto
villaverde alto
valdefuentes
valdefuentes
villaverde alto
villaverde alto
valdefuentes
valdefuentes
valverde

C:\Users\extas\AppData\Local\Temp\ipykernel_21052\3526477367.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '171' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df17.at[idx, 'cod_barrio'] = valores.get('cod_barrio')


In [74]:
# Filtrar filas donde 'cod_barrio' no es nulo y 'cod_distrito' es nulo
filas = df17.query("cod_barrio.isnull() ")

# Iterar sobre las filas filtradas
for idx, fila in filas.iterrows():
    if pd.notna(fila['zona']):
        bar = unidecode(fila['zona'])
        if bar=='villaverdealto,cascohistoricodevillaverde':           
            bar='villaverde alto'
        if bar=='loscarmenes':
            bar='los carmenes'
        if bar=='angeles':
            bar='los angeles'
        if bar=='cascohistoricodevicalvaro':
            bar='casco historico de vicalvaro'
        if bar=='elsalvador':
            bar='salvador'
        if bar=='Malasana':
            bar='universidad'
        if bar=='Ventilla':
            bar='almenara'
      #print (fila['zona'])
        valores = obtener_valores_por_columna(dfbarrios, 'distrito_n', bar)
        print(bar)    
        if valores:  
           # if bar=='valdefuentes':
            print(bar)         
            #if pd.isna(df17.at[idx, 'cod_distrito']): 
            n= valores.get('cod_distrito')
            df17.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
            #if pd.isna(df17.at[idx, 'distrito']): 
            df17.at[idx, 'distrito'] = valores.get('distrito')
           

Villaverde
Villaverde
Centro
Centro
PuentedeVallecas
CiudadLineal
Centro
Centro
Vicalvaro
Vicalvaro
Vicalvaro
Vicalvaro
Moratalaz
Moratalaz
CampodelasNaciones
Vallecas
Carabanchel
Carabanchel
PuentedeVallecas
Valdebebas
Salamanca
Salamanca
PuertadeHierro
Arganzuela
Arganzuela
Centro
Centro
Tetuan
Tetuan
Centro
Centro
Arganzuela
Arganzuela
Tetuan
Tetuan
Centro
Centro
CascoHistoricodeBarajas
CiudadLineal
PuentedeVallecas
Centro
Centro
CiudadLineal
Bernabeu
Arganzuela
Arganzuela
Salamanca
Salamanca
Centro
Centro
CiudadLineal
Bernabeu
Chueca
Madrid
PauCarabanchel
Salamanca
Salamanca
Centro
Centro
GranVia
Salamanca
Salamanca
Centro
Centro
Chamartin
Chamartin
Centro
Centro
Centro
Centro
ArturoSoria
Carabanchel
Carabanchel
Centro
Centro
LasMercedes
Madrid
ArturoSoria
Chamartin
Chamartin
Retiro
Retiro
CascoHistoricodeBarajas
LasMercedes
Moncloa
Bernabeu
Salamanca
Salamanca
ArturoSoria
Sanchinarro
Retiro
Retiro
Chamartin
Chamartin
Chamberi
Chamberi
Vallecas
Centro
Centro
Villaverde
Villaverde
P

In [75]:
df17.query("cod_barrio.isnull() ")[['url','zona','distrito','cod_distrito','barrio','cod_barrio','descripcion_extendida']]

,url,zona,distrito,cod_distrito,barrio,cod_barrio,descripcion_extendida
9535,https://www.pisosmadrid.com.es/propiedad/3863841-piso-en-venta-en-madrid,NaN,Villaverde,17.00,NaN,NaN,PISO EN MADRID Vivienda situada en Avenida Palomeras en Madrid VIVIENDA CON INCIDENCIAS INMUEBLE SIN POSESION El estado de ocupado yo los impedimentos para que el inmueble pueda ser visitado libremente por los t
9539,https://www.pisosmadrid.com.es/propiedad/3866311-piso-en-venta-en-villaverde-madrid,Villaverde,Villaverde,17.00,NaN,NaN,OPORTUNIDAD IDEAL INVERSORES EL INMUEBLE SE ENCUENTRA OCUPADO por persona sin justo titulo Debido al estado ocupacional del inmueble no se pueden realizar visitas al mismoVivienda exterior de tres dormitorios s
9546,https://www.pisosmadrid.com.es/propiedad/3867131-piso-en-venta-en-madrid,NaN,Villaverde,17.00,NaN,NaN,Piso 3 dormitorios 1 banos 0 garajes Para reformar en Madrid Madrid inmueble sin posesion con ocupantes no es posible visitar se vende como inversionbrbrbrbrSe trata de una Vivienda situada en un edifi
9558,https://www.pisosmadrid.com.es/propiedad/3867157-piso-en-venta-en-centro-madrid,Centro,Centro,1.00,NaN,NaN,Piso 3 dormitorios 1 banos 0 garajes Para reformar en Madrid Madrid inmueble ocupado por tercero con titulo vencido no es posible visitar se vende como inversion si desea mas informacion no dude en contactar brc
9562,https://www.pisosmadrid.com.es/propiedad/3872239-piso-en-venta-en-madrid,NaN,Ciudad Lineal,15.00,NaN,NaN,DP2020 vende piso en edificio residencial con ascensor ubicado en la localidad de MadridConsta de una superficie de 102m2 bien distribuida 3 dormitorios amplios 2 banos un amplio y luminoso saloncomedor con salida a
...,...,...,...,...,...,...,...
10396,https://www.pisosmadrid.com.es/propiedad/4083308-piso-en-venta-en-arganzuela-madrid,Arganzuela,Arganzuela,2.00,NaN,NaN,Te cuento sobre una propiedad que no te dejara indiferente Te presentamos una fantastica vivienda en primera planta con amplios miradores y ventanales que llenan cada rincon de luz natural gracias a su orientacion
10406,https://www.pisosmadrid.com.es/propiedad/4084726-piso-en-venta-en-centro-madrid,Centro,Centro,1.00,NaN,NaN,DS Realtors ofrece SIN HONORARIOS AL COMPRADOR esta fantastica vivienda con reforma integral a estrenar con calidades de alta gama materiales de primera calidad y decorada por interioristaDS Realtors presenta un fa
10411,https://www.pisosmadrid.com.es/propiedad/4085127-apartamento-en-venta-en-madrid,NaN,Puente de Vallecas,13.00,NaN,NaN,Te presentamos este fantastico estudio en la calle Soria 5 perfecto para quienes buscan amplitud y comodidad Con una superficie generosa y techos de 3 metros de altura este estudio ofrece una sensacion de espacio unic
10422,https://www.pisosmadrid.com.es/propiedad/4086493-piso-en-venta-en-madrid,NaN,Puente de Vallecas,13.00,NaN,NaN,Vivienda situada en Calle Transmonte en Madrid capital VIVIENDA CON INCIDENCIAS INMUEBLE SIN POSESION El estado de ocupado yo los impedimentos para que el inmueble pueda ser visitado libremente por los tecnicos


In [76]:
#como en el origen está bien, traigo los valores de pisos madrid
dfInd1limpio= pd.read_csv('../02-datos-limpios/dUnionSilvia23122024.csv',index_col=False)

In [77]:
dfInd1limpio.query("cod_barrio.isnull()")[['distrito','cod_distrito','barrio','cod_barrio','read_all']]

,distrito,cod_distrito,barrio,cod_barrio,read_all


In [78]:
# filtro para encontrar los valores iguales a 0 
df17[df17["cod_distrito"]== 0]

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,€_comunidad_mes,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,m2_utiles,vista_zona,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,orientacion_este,orientacion_norte,"orientacion_norte,_sur",orientacion_oeste,orientacion_sur,"orientacion_sur,_este",sistema_alarma,n.visitas,grupo_cocina


In [79]:
df17[df17["cod_barrio"]== 0]

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,€_comunidad_mes,acceso_minusvalido,orientacion,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,m2_utiles,vista_zona,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,orientacion_este,orientacion_norte,"orientacion_norte,_sur",orientacion_oeste,orientacion_sur,"orientacion_sur,_este",sistema_alarma,n.visitas,grupo_cocina


In [80]:

# Muestreo de valores unicos 
df17["zona"].unique()
# Muestreo de valores unicos 
df17["calle"].unique()


array(['calle de nunez de balboa 2', nan, 'plaza de italia',
       'calle de juan ramon jimenez 8',
       'avenida de alberto de alcocer 12', 'calle alberto alcocer 10',
       'calle orense', 'calle alcala', 'calle de villaviciosa 4',
       'calle principe de vergara', 'calle infantas',
       'calle de moralzarzal, madrid', 'calle velazquez',
       'calle general pardinas', 'calle san juan de la salle',
       'calle jose lazaro galdiano', 'avenida alberto alcocer',
       'calle alcantara', 'calle castello', 'calle antonio grilo',
       'calle hermosilla', 'calle felix boix', 'calle juan de austria',
       'calle colon', 'avenida burgos', 'calle pastora imperio',
       'calle academia', 'calle odonnell', 'calle de castello, 42',
       'calle pedro de valdivia', 'calle arenal de maudes',
       'calle santa engracia', 'calle ecija',
       'calle raimundo fernandez villaverde', 'calle mayor',
       'calle don ramon de la cruz', 'calle de dulce chacon',
       'calle jose aba

In [81]:

# Muestreo de valores totales por cada valor unico
df_131["consumoce_ano"].value_counts()
#  Muestreo de nulos totales 
df_131["consumoce_ano"].isna().sum()
# Muestreo de los valores unicos 
df_131 ["letrace"].unique()
# Muestreo de los valores unicos
df_131["emisiones_co2"].unique()
# Muestreo aleatorios de valores en dos columnas 
df_131[["letrace","emisiones_co2"]].sample(10)
#  Muestreo de valores unicos 
df_131["tipologia"].unique()
# Muestreo de valores unicos
df_131["propiedad_completa"].unique()
# Muestreo de valores unicos 
df_131["estado"].unique()
# Muestreo de valores unicos 
df_131["amueblado"].unique()
#  Seleccion de columnas y filtro por indice
filtro_planta= df_131[["planta","url"]]
filtro_planta["url"][7762]
# Muestreo de valores aleatorios
df_131["planta"].sample(10)
#  Muestreo de valores unicos 
df_131["habitaciones"].unique()
#  Muestreo de valores unicos
df_131["dormitorios"].unique()
#  Filtro de valores mayor o igual a 20  
dormitorios_filtro=df_131[df_131["dormitorios"]>=20]
dormitorios_filtro["url"]
#  Muestreo de valores unicos 
df_131["banos"].unique()
#  Filtro de valores igual a 397.0 ....Aqui se puede ver claramente que en la descripcion menciona 3 baños y en la columna de banos aparecen 397.0.
banos_filtro= df_131[df_131["banos"]=="397.0"] 
banos_filtro
# filtro por url
banos_filtro["url"]
#  Filtro directo al valor "278.0" . Entrando al link se puede apreciar que el valor en si mismo no hace referencia a la cantidad de banos sino al espacio
# total del inmueble o llamado el pagina como superficie construida.
banos_filtro= df_131[df_131["banos"]=="278.0"] 
banos_filtro
# Muestreo de valores unicos 
df_131["balcon"].unique()
# Muestreo de valores unicos
df_131["terraza"].unique()
# Muestreo de valores unicos
df_131["garaje"].unique()
# Muestreo de valores aleatorios en dos columnas
df_131[["garaje","url"]].sample(10)
# Muestreo de valores unicos
df_131["trastero"].unique()
# Muestreo de valores unicos
df_131["ascensor"].unique()
# Muestreo de valores unicos 
df_131["porteria"].unique()

array(['Portero media jornada', nan, 'Portero todo el día'], dtype=object)

In [82]:
df17['cantidad_visitas'] = np.where((df17['cantidad_visitas'].isnull()) | (df17['cantidad_visitas'] == 0), df17['n.visitas'], df17['cantidad_visitas'])


In [83]:
df17['cantidad_visitas'].value_counts(dropna=False)

cantidad_visitas
NaN        8078
30.00       153
40.00       139
20.00       130
50.00        98
           ... 
2279.00       1
322.00        1
144.00        1
756.00        1
3364.00       1
Name: count, Length: 1101, dtype: int64

In [84]:
# Guardar el DataFrame en un archivo CSV sin incluir el índice
#df.to_csv("dfpisosCompleto17012025.csv", index=False)


In [85]:
df17["porteria"].value_counts(dropna=False)

porteria
NaN                      10275
Portero todo el día        148
Portero media jornada        4
Name: count, dtype: int64

# revision columnas

## terrazas , terraza

In [88]:
df17['terrazas'].describe()

count                       44
unique                       2
top       Numero de terrazas 1
freq                        31
Name: terrazas, dtype: object

In [89]:
df17['terraza'].describe()

count   6071.00
mean       0.32
std        0.47
min        0.00
25%        0.00
50%        0.00
75%        1.00
max        1.00
Name: terraza, dtype: float64

In [90]:
df17['terrazas'].unique()

array([nan, 'Numero de terrazas 1', 'Numero de terrazas 2'], dtype=object)

In [91]:
df17['terraza'].unique()

array([ 1., nan,  0.])

In [92]:
df17["terraza"].value_counts(dropna=False)

terraza
NaN     4356
0.00    4122
1.00    1949
Name: count, dtype: int64

In [93]:
# Función personalizada para asignar valores a la columna 'terraza'
def asignar_terraza(fila):
    if fila['terrazas'] == 'Numero de terrazas 1':
        return 1
    elif fila['terrazas'] == 'Numero de terrazas 2':
        return 2
    else:
        return fila['terraza']

# Aplicar la función personalizada a la columna 'terraza'
df17['terraza'] = df17.apply(asignar_terraza, axis=1)


In [94]:
df17["terraza"].value_counts(dropna=False)

terraza
NaN     4312
0.00    4122
1.00    1980
2.00      13
Name: count, dtype: int64

In [95]:
df17['terraza'].unique()

array([ 1., nan,  0.,  2.])

In [96]:


# Supongamos que df17 es tu DataFrame
#df17['porteria'] = df17['porteria'].replace('0', np.nan)
df17['calefaccion'] = df17['calefaccion'].replace('0.0', np.nan)
df17['calefaccion'] = df17['calefaccion'].replace('Climatización', 'climatizacion')
df17['calefaccion'] = df17['calefaccion'].replace('Climatizaciòn', 'climatizacion')
df17['tiene_armario'] = df17['tiene_armario'].replace(0.0, np.nan)
df17.loc[df17['web'].isnull(), 'web'] = 'Indomio.com'
df17['tiene_piscina'] = df17['tiene_piscina'].replace(0, np.nan)


df17['porteria'] = df17['porteria'].replace('1', 'Portero todo el día')


## calefaccion

In [340]:
df17["calefaccion"].value_counts(dropna=False)

calefaccion
NaN                                                   6889
Central                                                917
Individual                                             622
1.0                                                    578
Gas natural                                            520
Calefaccion Gas Natural                                254
Calefaccion Electrica                                  126
Eléctrica                                              113
Calefaccion Individual                                 104
Calefaccion Central                                     87
Individual, alimentación eléctrica                      33
Individual, por radiadores, de gas                      28
Individual - gas natural                                18
Central, por radiadores, de gas                         16
Suelo radiante                                          11
Individual, por radiadores                              10
Gasoil                                      

In [97]:


# Palabras clave para clasificar
central_keywords = ["central", "comunitaria", "comunitario"]
individual_keywords = ["individual", "particular", "chimenea", "bomba", "termo", "climatizacion"]
electric_keywords = ["electrica", "bomba", "climatización", 'termo electrico','climatizacion']
gas_keywords = ["gas", "propano", "g n", "g/n"]  # Asegurarse de que 'g n' y 'g/n' estén bien escritos
gasoil_keywords = ["gasóil","gasoil",'gasoil',"Gasoil"]  # Añadir "gasoil" aquí
renovables_keywords = ["aerotermia", "geotermia", "chimenea"]
suelo_radiante_keywords=["suelo_radiante", "suelo"]
#radiadores_keywords=["radiadores"]
aire_keywords=["bomba","climatización",'climatizacion','conducto']
desconocido_keywords=['chimenea', 'termo electrico']
#lo mas extendido son los radiadores por eso despues de pasar , suelo radiante y aire pongo a todos radiadores salvo a chimenea y termo electrico que no lo se
radiadores_keywords=['radiadores','Gas natural', 'G/n', 'Gas particular', 'Central, alimentación eléctrica', 'Agua caliente y calefacción central con contador i', 'Comunitario', 'Individual con gas natural', 'Individual, de gasóil', 'Comunitaria', 'Calefaccion Propano', 'Calefaccion Gas Natural', 'Central, por aire, de gas', 'Gas natural y biomasa', 'Centralizada', 'Eléctrica', 'Calefaccion Individual', 'Calefaccion individual','Contadores individuales', '1.0', 'Individual, alimentación eléctrica', 'Aerotermia', 'Individual - gas natural', 'Individual', 'Calefaccion Aerotermia', 'Gasoil', 'Central individualizada', 'Individual, por aire, alimentación eléctrica', 'Geotermia y aerotermia', 'Calefaccion comunitaria', 'Calefaccion Central', 'Central, de gas', 'Calefaccion Electrica', 'Calefacción central individualizada', 'Central, de gasóil', 'Individual, de gas', 'Central']

# Inicializar conjuntos para valores únicos
central = set()
individual = set()
electrica = set()
gas = set()
gsoil = set()
renovables = set()
suelo_radiante = set()
radiadores = set()
aire = set()
sin_clasificar = set()
sin_clasificar_energia = set()
sin_clasificar_medio = set()
radiadores=set()
suelo_radiante=set()


# Clasificar categorías
for categoria in df17["calefaccion"]:
    categoria_normalized = normalize_text(str(categoria))  # Normalizar texto
    
 
    
    # Clasificación por tipo (central o individual)
    if any(keyword in categoria_normalized for keyword in central_keywords):
        central.add(categoria)
    elif any(keyword in categoria_normalized for keyword in individual_keywords):
        individual.add(categoria)
    else:
        sin_clasificar.add(categoria)
    
    # Clasificación por energía (eléctrica, gas, gasoil)
    if any(keyword in categoria_normalized for keyword in electric_keywords):
        electrica.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gasoil_keywords):
        gsoil.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gas_keywords):
        gas.add(categoria)
      
   
    elif any(keyword in categoria_normalized for keyword in renovables_keywords):
        renovables.add(categoria)
    else:
        sin_clasificar_energia.add(categoria)

    # Clasificación por medio (suelo radiante, radiadores, aire)
    
   
    if any(keyword in categoria_normalized for keyword in aire_keywords):
        aire.add(categoria)    
    elif any(keyword in categoria_normalized for keyword in suelo_radiante_keywords):
        suelo_radiante.add(categoria)
    elif any(keyword in categoria_normalized for keyword in desconocido_keywords):
        sin_clasificar_medio.add(categoria)
  #  else: any(keyword in categoria_normalized for keyword in radiadores_keywords):
    else: 
        radiadores.add(categoria)
    

# Convertir los conjuntos a listas
central = list(central)
individual = list(individual)
sin_clasificar = list(sin_clasificar)
electrica = list(electrica)
gas = list(gas)
gasoil = list(gsoil)
renovables = list(renovables)
sin_clasificar_energia = list(sin_clasificar_energia)
aire=list(aire)
radiadores=list(radiadores)
suelo_radiante=list(suelo_radiante)
sin_clasificar_medio=list(sin_clasificar_medio)
# Imprimir resultados
print("Central:", central)
print("Individual:", individual)
print("No clasificado:", sin_clasificar)
print("Electrica:", electrica)
print("Gas:", gas)
print("Gasoil:", gasoil)
print("Renovables:", renovables)
print("No clasificado en energía:", sin_clasificar_energia)
print("Suelo Radiante:", suelo_radiante)
print("Radiadores:", radiadores)
print("Aire:", aire)
print("No clasificado en medio:", sin_clasificar_medio)


Central: ['Central', 'Comunitario', 'Central, por aire, de gas', 'Comunitaria', 'Central, suelo radiante, de gas', 'Central, por radiadores, de gas', 'Central, de gas', 'Central, por radiadores, de gasóil', 'Central, alimentación eléctrica', 'Calefaccion comunitaria', 'Central, por radiadores', 'Centralizada', 'Central, suelo radiante', 'Calefaccion Central', 'Central individualizada', 'Calefacción central individualizada', 'Central, de gasóil', 'Agua caliente y calefacción central con contador i']
Individual: ['Individual, por radiadores, de gas', 'Individual, de gas', 'Individual, por radiadores, de gasóil', 'Individual, por radiadores, alimentación eléctrica', 'Individual, suelo radiante, de gas', 'Individual - gas natural', 'Individual, por aire, alimentación eléctrica', 'Individual', 'climatizacion', 'Individual, de gasóil', 'Individual, suelo radiante', 'Chimenea', 'Individual, alimentación eléctrica', 'Por climatizacion', 'Bomba frio calor', 'Calefaccion individual', 'Climatizac

In [98]:
df17["ano_construccion"].value_counts(dropna=False)

ano_construccion
NaN                     6942
Más de 50 años          1018
Entre 30 y 50 años       205
 Mas de 50 anos          131
1900.0                   115
                        ... 
 Entre 10 y  15 Anos       1
 1949                      1
 1943                      1
 1928                      1
1944.0                     1
Name: count, Length: 188, dtype: int64

In [99]:
df17["plantas_edificio"].value_counts(dropna=False)

plantas_edificio
NaN      10211
4.00        40
6.00        40
3.00        32
7.00        25
5.00        24
8.00        16
2.00        13
10.00        9
9.00         8
1.00         5
12.00        2
23.00        1
11.00        1
Name: count, dtype: int64

In [100]:
df17.columns = df17.columns.str.strip()

In [345]:
df17.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio',
       ...
       'cantidad_dormitorios', 'orientacion_este', 'orientacion_norte',
       'orientacion_norte,_sur', 'orientacion_oeste', 'orientacion_sur',
       'orientacion_sur,_este', 'sistema_alarma', 'n.visitas', 'grupo_cocina'],
      dtype='object', length=110)

In [101]:
df17["plantas_edificio"].value_counts(dropna=False)

plantas_edificio
NaN      10211
4.00        40
6.00        40
3.00        32
7.00        25
5.00        24
8.00        16
2.00        13
10.00        9
9.00         8
1.00         5
12.00        2
23.00        1
11.00        1
Name: count, dtype: int64

## aire acondicionado

In [347]:
df17["aire_acondicionado"].value_counts(dropna=False)

aire_acondicionado
NaN                                                   5920
0                                                     1036
0.0                                                    844
Individual                                             626
Frío                                                   477
1                                                      432
Individual, frío/calor                                 418
Frío y calor                                           319
Individual, frío                                       125
Splites repartidos                                      70
1.0                                                     37
Independiente                                           26
Frío/calor                                              20
Centralizada                                            16
Suelo refrigerante                                      10
Frío-calor                                               7
Preinstalación                       

In [348]:
df17["aire_acondicionado"].unique()


array([nan, 'Preinstalación', 'Individual, frío/calor',
       'Individual, frío', 'Individual', 'Frío/calor',
       'Centralizada, frío', 'Centralizada, frío/calor',
       'Centralizada, calor', 'Centralizada', 'Frío', 'Frío y calor',
       'Splites repartidos', 'Por conductos en toda la casa',
       'Aerotermia', 'Independiente', 'Con bomba de calor', 'A/a f/c',
       'Suelo refrigerante', 'Preinstalado', 'Frío-calor',
       'Suelo radiante refrescante', 'Sistema air zone',
       'Climatizacion por conductos frio y calor',
       'Frió/calor por conductos', 'Central', 'Suelo radiante',
       'Tiene la instalción hecha en toda la casa por cond',
       'Geotermia y aerotermia', 'En toda la casa, por conductos',
       'Aire frio/ calor por conductos/suelo refrescante',
       'No disponible', '1', '0', '0.0', '1.0'], dtype=object)

In [102]:
# Listas de valores para cada grupo
eficientes_val_columna = [
    'Aerotermia', 'Geotermia y aerotermia', 'Suelo radiante refrescante', 'Suelo refrigerante',
    'Sistema air zone', 'Climatizacion por conductos frio y calor', 'Frió/calor por conductos', 
    'Suelo radiante', 'Aire frio/ calor por conductos/suelo refrescante', 'Por conductos en toda la casa', 
    'Tiene la instalción hecha en toda la casa por cond', 'En toda la casa, por conductos'
]

normales_val_columna = [
    'Individual, frío/calor', 'Individual, frío', 'Individual', 'Frío/calor', 'Centralizada, frío', 
    'Centralizada, frío/calor', 'Centralizada, calor', 'Central', 'Frío', 'Frío y calor', 
    'Splites repartidos', 'Independiente', 'Con bomba de calor', 'A/a f/c', 
    'Aire frio/calor por conductos/suelo refrescante', '1', '1.0'
]

sin_aire_val_columna = ['Preinstalación', 'Preinstalado', 'No disponible', '0', '0.0']

# Función para clasificar los sistemas
def clasificar_sistema(valor):
    if valor in eficientes_val_columna:
        return 'Eficiente'
    elif valor in normales_val_columna:
        return 'Normal'
    elif valor in sin_aire_val_columna:
        return 'Sin Aire'
    else:
        return 'Desconocido'  # Si el valor no coincide con ninguno de los grupos




In [103]:
# Crear una nueva columna en el DataFrame
df17['clasificacion_aire'] = df17['aire_acondicionado'].apply(clasificar_sistema)


In [104]:
df17["jardin"].unique()

array([nan, 'Privado', 'Sin jardín', 'Comunitario', '0.0', '1.0'],
      dtype=object)

In [105]:
# Ejemplo de uso con una sola columna
porcentaje = porcentaje_nulos(df17, 'plantas_edificio')
print(f"Porcentaje de valores nulos en 'plantas_edificio': {porcentaje.get('plantas_edificio', 'Column not found'):.2f}%")


Porcentaje de valores nulos en 'plantas_edificio': 97.93%


In [106]:

# Ejemplo de uso
columnas = ["inmueble_ingresos", "alquiler_opcion_a_compra", "disponibilidad","nuda_propiedad","sup_comercial"]  # Reemplaza con las columnas de tu DataFrame
porcentaje_resultados = porcentaje_nulos(df17, columnas)

# Mostrar los resultados
for columna, porcentaje in porcentaje_resultados.items():
    print(f"Porcentaje de nulos en {columna}: {porcentaje:.2f}%")

Porcentaje de nulos en inmueble_ingresos: 99.93%
Porcentaje de nulos en alquiler_opcion_a_compra: 99.97%
Porcentaje de nulos en disponibilidad: 95.72%
Porcentaje de nulos en nuda_propiedad: 99.98%
Porcentaje de nulos en sup_comercial: 96.58%


In [354]:
df17["disponibilidad"].value_counts(dropna=False)

disponibilidad
NaN      9981
Libre     446
Name: count, dtype: int64

La columna disponibilidad no aporta nada

In [355]:
df17["alquiler_opcion_a_compra"].value_counts(dropna=False)

alquiler_opcion_a_compra
NaN     10424
1.00        3
Name: count, dtype: int64

In [356]:
df17.query("(inmueble_ingresos == 1) | (alquiler_opcion_a_compra == 1) | (nuda_propiedad == 1)").count()


title                    12
url                      12
precio                   12
precio_anterior           0
descuento                 0
                         ..
orientacion_sur,_este     0
sistema_alarma            2
n.visitas                 0
grupo_cocina             12
clasificacion_aire       12
Length: 111, dtype: int64

## sup_comercial

In [ ]:
df17[df17['sup_comercial'].notnull()].head()

In [358]:
df17['tipo_inmueble'].value_counts(dropna=False)

tipo_inmueble
NaN          9863
económico     276
medio         171
señorial      117
Name: count, dtype: int64

In [359]:
df17['tiene_armario'].value_counts(dropna=False)

tiene_armario
NaN     9725
1.00     702
Name: count, dtype: int64

In [360]:
df17['cancha_tenis'].value_counts(dropna=False)

cancha_tenis
NaN     7088
0.00    3336
1.00       3
Name: count, dtype: int64

In [361]:
df17['chimenea'].value_counts(dropna=False)

chimenea
NaN     6207
0.00    4169
1.00      51
Name: count, dtype: int64

In [ ]:
df17[df17['chimenea'] == 1][['calefaccion', 'chimenea']]

In [363]:
df17['fibra_optica'].value_counts(dropna=False)

fibra_optica
NaN     7088
0.00    3301
1.00      38
Name: count, dtype: int64

In [364]:
df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     9338
1.00    1089
Name: count, dtype: int64

In [365]:
df17['carpinteria_exterior_doble_vidrio/pvc'].value_counts(dropna=False)

carpinteria_exterior_doble_vidrio/pvc
NaN     7088
0.00    3323
1.00      16
Name: count, dtype: int64

## cocina

In [366]:
df17['cocina'].value_counts(dropna=False)

cocina
NaN                                   8835
Cocina equipada                        590
Cocina independiente                   318
Cocina amueblada                       190
Independiente                          131
                                      ... 
Equipada con excelentes calidades        1
Con entrada independiente                1
Independiente amueblada y equipada       1
Independiente con despensa               1
Amueblada reformada                      1
Name: count, Length: 88, dtype: int64

In [367]:
df17['cocina'].unique()

array([nan, 'Cocina abierta', 'Cocina independiente',
       'Cocina abierta amueblada', 'Cocina independiente amueblada',
       'Cocina americana', 'Amueblada con electrodomésticos',
       'Cocina equipada', 'Americana', 'Independiente y equipada',
       'Amueblada sin electrodomésticos', 'Cocina amueblada',
       'Independiente', 'Amueblada, equipada', 'Individual',
       'Amueblada y en isla', 'Equipada', 'Amueblada',
       'Independiente con zona de servicio',
       'Independiente - amueblada con elec.', 'Independiente con office',
       'Amplia', 'Amueblada y equipada con electrodomésticos',
       'Pequeña cocina integrada', 'Americana, amueblada y equipada.',
       'Amueblada con electrodomésticos, caldera nueva',
       'Amueblada con electrodomesticos',
       '0ffice,habitacion ,baño y terraza',
       'Completamente equipada abierta al salón',
       'Independiente con office, dormi. Servic. Con baño',
       'En office amplia completamente equipada',
       'Co ofi

In [368]:
df17.query("amueblado=='Sólo cocina amueblada' and cocina.notnull()")[['amueblado','cocina']]

,amueblado,cocina
483,Sólo cocina amueblada,Cocina abierta amueblada
1249,Sólo cocina amueblada,Cocina independiente amueblada
1701,Sólo cocina amueblada,Cocina independiente amueblada
1751,Sólo cocina amueblada,Cocina independiente amueblada
2742,Sólo cocina amueblada,Cocina independiente amueblada
9458,Sólo cocina amueblada,Cocina independiente amueblada


In [107]:
#pongo el valor de cocina_1 en cocina y elimino el campo cocina_1
#df17.loc[df17['cocina_1'].notnull(), 'cocina'] = df17['cocina_1']
#el campo cocina_equipada que aparece en redpiso lo integro en cocina y luebo elimino el campo
#df17.loc[df17['cocina_equipada']==1, 'cocina'] = 'Cocina equipada'
df17.loc[df17['cocina']=='Francesa', 'cocina'] = 'independiente y amueblada'
df17.loc[(df17['amueblado'] == 'Sólo cocina amueblada') & (df17['cocina'].notnull()), 'cocina'] = df17['cocina'] + ' amueblada'
df17.loc[df17['cocina']=='Equipafda americana', 'cocina'] = 'Equipada americana'

#df17.drop(columns=['cocina_1'], inplace=True)
#df17.drop(columns=['cocina_equipada'], inplace=True)


In [108]:
def agrupar_cocinas(tipo):
    categorias = []
    if pd.isnull(tipo):
        categorias.append('sin_informacion')
    elif isinstance(tipo, str):
        if 'abierta' in tipo.lower() or 'abierto' in tipo.lower() or 'americana' in tipo.lower() or 'integrada' in tipo.lower():
            categorias.append('abierta')
        if 'independiente' in tipo.lower():
            categorias.append('independiente')
        if 'amplia' in tipo.lower():
            categorias.append('amplia')
        if 'isla' in tipo.lower():
            categorias.append('con isla')
        if 'amueblada' in tipo.lower():
            categorias.append('amueblada')
        if 'con electrodomesticos' in tipo.lower() or 'equipada' in tipo.lower() or 'con elec.' in tipo.lower():
            categorias.append('equipada')       
        if 'sin amueblar' in tipo.lower():
            categorias.append('no amueblada')
        if 'reformada' in tipo.lower():
            return 'reformada'
        elif 'a reformar' in tipo.lower() or 'necesita reforma' in tipo.lower():
            return 'Sin reforma'
        if not categorias:
            categorias.append('sin categoria')
        return ', '.join(categorias)
    else:
        return 'error tipo campo no texto'
    return ', '.join(categorias)


In [109]:
# Aplica la función de agrupación
df17['grupo_cocina'] = df17['cocina'].apply(agrupar_cocinas)

print(df17[['cocina', 'grupo_cocina']])


               cocina     grupo_cocina
0                 NaN  sin_informacion
1      Cocina abierta          abierta
2                 NaN  sin_informacion
3                 NaN  sin_informacion
4                 NaN  sin_informacion
...               ...              ...
10422             NaN  sin_informacion
10423             NaN  sin_informacion
10424             NaN  sin_informacion
10425             NaN  sin_informacion
10426             NaN  sin_informacion

[10427 rows x 2 columns]


In [372]:
df17.query("cocina.notnull()")['cocina'].unique()

array(['Cocina abierta', 'Cocina independiente',
       'Cocina abierta amueblada amueblada',
       'Cocina independiente amueblada amueblada', 'Cocina americana',
       'Amueblada con electrodomésticos', 'Cocina equipada', 'Americana',
       'Independiente y equipada', 'Amueblada sin electrodomésticos',
       'Cocina amueblada', 'Independiente', 'Amueblada, equipada',
       'Individual', 'Amueblada y en isla', 'Equipada', 'Amueblada',
       'Independiente con zona de servicio',
       'Independiente - amueblada con elec.', 'Independiente con office',
       'Amplia', 'Amueblada y equipada con electrodomésticos',
       'Pequeña cocina integrada', 'Americana, amueblada y equipada.',
       'Amueblada con electrodomésticos, caldera nueva',
       'Amueblada con electrodomesticos',
       '0ffice,habitacion ,baño y terraza',
       'Completamente equipada abierta al salón',
       'Independiente con office, dormi. Servic. Con baño',
       'En office amplia completamente equipada',

In [373]:
df17.query("amueblado=='Sólo cocina amueblada'").count()

title                    71
url                      71
precio                   71
precio_anterior           0
descuento                 0
                         ..
orientacion_sur,_este     0
sistema_alarma            5
n.visitas                 0
grupo_cocina             71
clasificacion_aire       71
Length: 111, dtype: int64

In [374]:
df17['amueblado'].value_counts(dropna=False)

amueblado
NaN                       7531
0                         1467
0.0                        720
Sí                         293
No                         171
1.0                        161
Sólo cocina amueblada       71
Parcialmente amueblado      12
1                            1
Name: count, dtype: int64

In [375]:
## interior, exterior
df17['interior'].value_counts(dropna=False)

interior
0.00    7693
NaN     2641
1.00      93
Name: count, dtype: int64

In [376]:

df17['exterior'].value_counts(dropna=False)

exterior
0.00    7386
1.00    1807
NaN     1234
Name: count, dtype: int64

In [377]:
df17["vista_zona"].unique()

array([nan, 'Exterior'], dtype=object)

In [378]:
# Si el valor de la columna 'exterior' es nulo y la columna 'vista_zona' tiene el valor 'Exterior', poner en la columna 'exterior' un 1
df17.loc[df17['exterior'].isnull() & (df17['vista_zona'] == 'Exterior'), 'exterior'] = 1

# Eliminar la columna 'vista_zona'
df17.drop(columns=['vista_zona'], inplace=True)

In [379]:

df17['interior_y_exterior'].value_counts(dropna=False)

interior_y_exterior
NaN     7088
0.00    3330
1.00       9
Name: count, dtype: int64

In [380]:

df17.query("interior.isnull() & exterior.isnull() & interior_y_exterior.isnull()")["web"].unique()

array(['redpiso', 'pisosmadrid'], dtype=object)

## piscina

In [381]:


df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     9338
1.00    1089
Name: count, dtype: int64

In [382]:
df17['piscina'].value_counts(dropna=False)

piscina
 NaN                                          4804
 -                                            3058
0.00                                          1804
 0                                             310
1.00                                           141
 Comunitaria                                   139
 0.0                                            89
 Propia                                         75
 1                                               4
 Con salorium                                    1
 1 piscina de adultos y 1 piscina de niños       1
 1.0                                             1
Name: count, dtype: int64

In [383]:

#asignamos el valor tiene_piscina sin indicar el tipo
df17.loc[(df17['piscina'] != 0.0) & (df17['piscina'] != 0) & (df17['piscina'] != '-') & (df17['piscina'].notnull()), 'tiene_piscina'] = 1

In [384]:
df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     9249
1.00    1178
Name: count, dtype: int64

In [385]:

df17.query("tiene_piscina.notnull()")["web"].unique()

array(['Indomio.com', 'pisos.com', 'redpiso', 'pisosmadrid'], dtype=object)

## porton electrico, puerta blindada

In [386]:

## portn_electrico,puerta_blindad

columnas = ["porton_electrico", "puerta_blindada", "alarma","videoportero","lujosa","descripcion_extendida"]  
porcentaje_resultados = porcentaje_nulos(df17, columnas)


# Mostrar los resultados
for columna, porcentaje in porcentaje_resultados.items():
    print(f"Porcentaje de nulos en {columna}: {porcentaje:.2f}%")

Porcentaje de nulos en porton_electrico: 67.98%
Porcentaje de nulos en puerta_blindada: 67.98%
Porcentaje de nulos en alarma: 48.14%
Porcentaje de nulos en videoportero: 59.53%
Porcentaje de nulos en lujosa: 67.98%
Porcentaje de nulos en descripcion_extendida: 45.86%


In [387]:

df17['porton_electrico'].value_counts(dropna=False)

porton_electrico
NaN     7088
0.00    3323
1.00      16
Name: count, dtype: int64

In [388]:

df17['puerta_blindada'].value_counts(dropna=False)

puerta_blindada
NaN     7088
0.00    3243
1.00      96
Name: count, dtype: int64

In [389]:

df17['alarma'].value_counts(dropna=False)

alarma
0.00    5324
NaN     5020
1.00      83
Name: count, dtype: int64

In [390]:

df17.query("alarma==1 | puerta_blindada==1")[['precio','alarma','puerta_blindada']]

,precio,alarma,puerta_blindada
0,2000000.00,0.00,1.00
1,287000.00,0.00,1.00
17,450000.00,0.00,1.00
50,199990.00,0.00,1.00
76,176000.00,0.00,1.00
...,...,...,...
10320,1939000,1.00,NaN
10367,800000,1.00,NaN
10406,870000,1.00,NaN
10407,515000,1.00,NaN


In [391]:

df17['videoportero'].value_counts(dropna=False)

videoportero
NaN     6207
0.00    3874
1.00     346
Name: count, dtype: int64

In [392]:
## lujosa

In [393]:

df17['lujosa'].value_counts(dropna=False)

lujosa
NaN     7088
0.00    1833
1.00    1506
Name: count, dtype: int64

In [394]:

df17.query("videoportero==1").count()

title                    346
url                      346
precio                   346
precio_anterior            0
descuento                  0
                        ... 
orientacion_sur,_este      0
sistema_alarma             1
n.visitas                282
grupo_cocina             346
clasificacion_aire       346
Length: 110, dtype: int64

## descripcion_extendida

################ REVISAR #################

€_comunidad_mes

In [395]:
df17['€_comunidad_mes'].value_counts(dropna=False)

€_comunidad_mes
NaN                 9112
Más de 100 €         350
Entre 40 y 60 €      100
Entre 80 y 100 €      95
Entre 60 y 80 €       80
                    ... 
336                    1
175                    1
325                    1
525                    1
60 €.-                 1
Name: count, Length: 314, dtype: int64

In [396]:
df17['gastos_comunidad'].value_counts(dropna=False)

gastos_comunidad
NaN                10337
Comunidad 5000         9
€ 50/mes               5
€ 100/mes              5
Comunidad 4500         4
Comunidad 4000         3
€ 60/mes               3
€ 350/mes              3
€ 80/mes               3
€ 72/mes               2
€ 55/mes               2
€ 148/mes              2
€ 112/mes              2
€ 37/mes               2
€ 160/mes              2
Comunidad 3750         1
€ 26/mes               1
€ 125/mes              1
Comunidad 10000        1
Comunidad 18000        1
Comunidad 2000         1
€ 58/mes               1
€ 283/mes              1
€ 27/mes               1
€ 33/mes               1
€ 103/mes              1
€ 43/mes               1
€ 163/mes              1
Comunidad 9900         1
€ 30/mes               1
€ 180/mes              1
€ 280/mes              1
€ 404/mes              1
€ 109/mes              1
€ 45/mes               1
€ 230/mes              1
Comunidad 11000        1
Comunidad 23600        1
€ 95/mes               1
Comunida

In [397]:
df17.shape

(10427, 110)

In [398]:
df17.query("`€_comunidad_mes`.notnull() | gastos_comunidad.notnull()")[['web','gastos_comunidad','€_comunidad_mes']]

,web,gastos_comunidad,€_comunidad_mes
0,Indomio.com,NaN,100
1,Indomio.com,NaN,123
11,Indomio.com,NaN,55
12,Indomio.com,NaN,60
68,Indomio.com,NaN,258
...,...,...,...
9515,Indomio.com,€ 100/mes,NaN
9519,Indomio.com,€ 100/mes,NaN
9520,Indomio.com,€ 140/mes,NaN
9522,Indomio.com,€ 50/mes,NaN


In [399]:
df17.query("`€_comunidad_mes`.notnull() and  gastos_comunidad.isnull() ")[['web','gastos_comunidad','€_comunidad_mes']]

,web,gastos_comunidad,€_comunidad_mes
0,Indomio.com,NaN,100
1,Indomio.com,NaN,123
11,Indomio.com,NaN,55
12,Indomio.com,NaN,60
68,Indomio.com,NaN,258
...,...,...,...
7774,pisos.com,NaN,Más de 100 €
7781,pisos.com,NaN,Más de 100 €
7782,pisos.com,NaN,38
7784,pisos.com,NaN,Entre 20 y 40 €


In [400]:
df17.query(" gastos_comunidad.notnull() and `€_comunidad_mes`.isnull() ")[['web','url','gastos_comunidad','€_comunidad_mes']]

,web,url,gastos_comunidad,€_comunidad_mes
7800,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-los-rosales-villaverde-madrid-madrid-RP2372023110992,Comunidad 7400,NaN
7822,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-villaverde-madrid-madrid-RP2372024127521,Comunidad 4000,NaN
7884,redpiso,https://www.redpiso.es/inmueble/atico-en-venta-en-calle-san-clodoaldo-ventas-ciudad-lineal-madrid-madrid-RP162024125933,Comunidad 9900,NaN
7938,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-godella-san-cristobal-villaverde-madrid-madrid-RP2842024125361,Comunidad 5000,NaN
7945,redpiso,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-almendrales-almendrales-usera-madrid-madrid-RP392024129914,Comunidad 2000,NaN
...,...,...,...,...
9515,Indomio.com,https://www.indomio.es/anuncios/96986027/,€ 100/mes,NaN
9519,Indomio.com,https://www.indomio.es/anuncios/96967621/,€ 100/mes,NaN
9520,Indomio.com,https://www.indomio.es/anuncios/96979909/,€ 140/mes,NaN
9522,Indomio.com,https://www.indomio.es/anuncios/96696025/,€ 50/mes,NaN


In [401]:
#En redpiso los valores hay que dividirlos entre 100 al procesarlo se quito la coma que tenia en 20,00 ahora aparece con 2000
#pisos madrid y no tiene gastos de comunidad

In [402]:
df17.gastos_comunidad.unique()

array([nan, 'Comunidad 7400', 'Comunidad 4000', 'Comunidad 9900',
       'Comunidad 5000', 'Comunidad 2000', 'Comunidad 4500',
       'Comunidad 18000', 'Comunidad 10000', 'Comunidad 11000',
       'Comunidad 6500', 'Comunidad 3500', 'Comunidad 15000',
       'Comunidad 6900', 'Comunidad 3000', 'Comunidad 3750',
       'Comunidad 23600', 'Comunidad 6000', 'Comunidad 38000',
       '€ 175/mes', '€ 50/mes', '€ 37/mes', '€ 35/mes', '€ 112/mes',
       '€ 457/mes', '€ 95/mes', '€ 40/mes', '€ 111/mes', '€ 148/mes',
       '€ 160/mes', '€ 80/mes', '€ 138/mes', '€ 90/mes', '€ 120/mes',
       '€ 61/mes', '€ 451/mes', '€ 350/mes', '€ 43/mes', '€ 230/mes',
       '€ 125/mes', '€ 55/mes', '€ 100/mes', '€ 72/mes', '€ 58/mes',
       '€ 283/mes', '€ 27/mes', '€ 33/mes', '€ 26/mes', '€ 103/mes',
       '€ 163/mes', '€ 60/mes', '€ 30/mes', '€ 180/mes', '€ 280/mes',
       '€ 404/mes', '€ 109/mes', '€ 45/mes', '€ 140/mes'], dtype=object)

In [403]:
# Sustituir las palabras 'Comunidad', '€', y '/mes' por vacío en 'gastos_comunidad'
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('Comunidad', '')
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('€', '')
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('/mes', '')
# Eliminar espacios en blanco adicionales
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.strip()
 #Convertir solo los valores no nulos de 'gastos_comunidad' a tipo float
df17.loc[df17['gastos_comunidad'].notnull(), 'gastos_comunidad'] = df17.loc[df17['gastos_comunidad'].notnull(), 'gastos_comunidad'].astype(float)
# Dividir el valor de 'gastos_comunidad' entre 100 cuando 'web' es igual a 'redpiso'
df17.loc[(df17['web'] == 'redpiso') & (df17['gastos_comunidad'].notnull()), 'gastos_comunidad'] /= 100

# Dividir el valor de 'gastos_comunidad' entre 100 cuando 'web' es igual a 'redpiso'
# Convertir la columna 'gastos_comunidad' a tipo float
#df17['gastos_comunidad'] = pd.to_numeric(df17['gastos_comunidad'], errors='coerce')
#df17.loc[df17['web'] == 'redpiso', 'gastos_comunidad'] /= 100

In [404]:
df17.gastos_comunidad.unique()

array([nan, 74.0, 40.0, 99.0, 50.0, 20.0, 45.0, 180.0, 100.0, 110.0, 65.0,
       35.0, 150.0, 69.0, 30.0, 37.5, 236.0, 60.0, 380.0, 175.0, 37.0,
       112.0, 457.0, 95.0, 111.0, 148.0, 160.0, 80.0, 138.0, 90.0, 120.0,
       61.0, 451.0, 350.0, 43.0, 230.0, 125.0, 55.0, 72.0, 58.0, 283.0,
       27.0, 33.0, 26.0, 103.0, 163.0, 280.0, 404.0, 109.0, 140.0],
      dtype=object)

In [405]:
# Obtener los valores únicos de la columna '€_comunidad_mes'
df17['€_comunidad_mes'].unique()


array(['100', '123', nan, '55', '60', '258', '50', '92', '27', '90',
       '132', '128', '113', '800', '180', '1.300', '65', '625', '34',
       '150', '308', '146', 'Ningún gasto de comunidad', '310', '350',
       '110', '96', '120', '210', '70', '1.000', '250', '84', '247', '3',
       '56', '450', '40', '200', '75', '190', '166', '15', '45', '228',
       '72', '9', '294', '30', '160', '430', '95', '104', '63', '13',
       '85', '119', '300', '168', '340', '99', '164', '130', '241', '167',
       '5', '33', '111', '420', '206', '500', '10', '14', '540', '750',
       '148', '147', '386', '17', '76', '230', '205', '46', '47', '400',
       '32', '12', '36', '82', '155', '303', '207', '486', '83', '80',
       '66', '98', '178', '140', '52', '64', '54', '42', '29', '88', '61',
       '1.600', '440', '170', '57', '105', '142', '255', '125', '600',
       '324', '67', '269', '285', '38', '39', '102', '270', '44', '103',
       '617', '552', '311', '457', '192', '327', '288', '316', '

In [406]:
# Usar comillas invertidas para nombres de columnas con caracteres especiales
df17.query("`€_comunidad_mes` == 'Comunidad: 320€ mensual  / ibi 1.040,96€ anual'")[["web"]]

,web
4773,pisos.com


#############revisar

#Tengo que ir al origen para separar los valores de gastos de comunidad y de ibi

In [407]:
# Reemplazar valores nulos en 'gastos_comunidad' con los valores de '€_comunidad_mes'
df17['gastos_comunidad'] = df17['gastos_comunidad'].fillna(df17['€_comunidad_mes'])
# Eliminar la columna '€_comunidad_mes'
df17.drop(columns=['€_comunidad_mes'], inplace=True)

In [408]:

# Reemplazar 'No tiene' y 'Ningún gasto de comunidad' por '0'
df17['gastos_comunidad'] = df17['gastos_comunidad'].replace({'No tiene': '0', 'Ningún gasto de comunidad': '0'})


In [409]:
df17['acceso_minusvalido'].value_counts(dropna=False)

acceso_minusvalido
 NaN    8710
 0      1054
0.00     582
1.00      59
 1        10
 Sí        7
 No        5
Name: count, dtype: int64

## acceso_minusvalidos

In [410]:
# Reemplazar 'Sí' por 1 y 'No' por 0 en la columna 'acceso_minusvalido'
df17['acceso_minusvalido'] = df17['acceso_minusvalido'].replace({'Sí': 1, 'No': 0})

# Renombrar la columna 'acceso_minusvalido' a 'acceso_discapacidad'
df17.rename(columns={'acceso_minusvalido': 'acceso_discapacidad'}, inplace=True)

## orientacion

In [411]:
df17["orientacion"].value_counts()

orientacion
Sur                                312
Este                               263
Orientacion Sur                    162
Orientacion Este                   152
Oeste                              148
Norte                              131
Orientacion Oeste                  125
Sureste                            105
Orientacion Norte                   80
Orientacion Sureste                 71
Orientacion Suroeste                59
Suroeste                            58
Orientacion Noroeste                53
Noreste                             52
Noroeste                            51
Orientacion Noreste                 51
Medio día                           10
-1                                   9
sur                                  8
este                                 7
norte                                4
oeste                                2
Nordeste                             2
A la montaña                         2
sur_este                             2
Este, noreste

In [412]:
# Eliminar 'Orientacion' y limpiar espacios adicionales
df17['orientacion'] = df17['orientacion'].str.replace('Orientacion', '').str.strip()

# Sustituir ',' y '-' por '_' y poner los valores en minúsculas en la columna 'orientacion'
df17['orientacion'] = df17['orientacion'].str.replace(',', '_').str.replace('/', '_').str.replace('|', '_').str.replace('y', '_').str.replace('-', '_').str.lower()

# Eliminar espacios adicionales entre palabras
df17['orientacion'] = df17['orientacion'].str.replace(r'\s+', '_', regex=True).str.strip()
df17['orientacion'] = df17['orientacion'].str.replace('__', '_')

# Reemplazar múltiples valores en la columna 'orientacion' por np.nan
df17['orientacion'] = df17['orientacion'].replace({
    '__':'_',
    '_1': np.nan,
    'medio_día': np.nan,
    'sur_en_la_terraza__dormitorios': np.nan,
    'vistas_al_palacio_de_parcent.': np.nan,
    'varias':np.nan
})

df17['orientacion'] = df17['orientacion'].str.replace('__', '_')



In [413]:
# Obtener los valores únicos de la columna 'orientacion' donde los valores no son nulos
df17.loc[df17['orientacion'].notnull(), 'orientacion'].unique()


array(['norte', 'sur_este', 'este', 'oeste', 'sur', 'norte_sur',
       'noroeste', 'sureste', 'suroeste', 'noreste', 'nordeste',
       'a_la_montaña', 'este_oeste', 'sur_este_norte',
       'este_noreste_suroeste', 'sur_oeste', 'oeste_este', 'norte_este',
       'este_oeste_norte', 'sudeste', 'sur_norte'], dtype=object)

In [414]:


# Procesar la columna 'orientacion'
for index, value in df17["orientacion"].items():
    # hasta los nan los convierte a string, recordar luego cambiar 'nan' por nan de nuevo
    value_str = str(value)
    if re.search(r'\bnorte\b', value_str):
        df17.at[index, "orientacion_norte"] = 1
    if re.search(r'\bsur\b', value_str):
        df17.at[index, "orientacion_sur"] = 1
    if re.search(r'\beste\b', value_str):
        df17.at[index, "orientacion_este"] = 1
    if re.search(r'\boeste\b', value_str):
        df17.at[index, "orientacion_oeste"] = 1
    if re.search(r'\bnoroeste\b', value_str):
        df17.at[index, "orientacion_norte"] = 1
        df17.at[index, "orientacion_oeste"] = 1

# Revisar las columnas orientacion_norte,_sur y orientacion_sur,_este
# y asegurarnos de que las combinaciones sean coherentes
if "orientacion_norte,_sur" in df17.columns:
    for index, value in df17["orientacion_norte,_sur"].items():
        if value == 1:
            df17.at[index, "orientacion_norte"] = 1
            df17.at[index, "orientacion_sur"] = 1

if "orientacion_sur,_este" in df17.columns:
    for index, value in df17["orientacion_sur,_este"].items():
        if value == 1:
            df17.at[index, "orientacion_sur"] = 1
            df17.at[index, "orientacion_este"] = 1


In [415]:
# Filtrar filas donde todas las columnas relevantes no sean nulas
filtered_df = df17[
    df17["orientacion"].notnull() |
    df17["orientacion_norte"].notnull() |
    df17["orientacion_sur"].notnull() |
    df17["orientacion_este"].notnull() |
    df17["orientacion_oeste"].notnull() |
    df17["orientacion_norte,_sur"].notnull() |
    df17["orientacion_sur,_este"].notnull()
]

# Mostrar las columnas deseadas
result = filtered_df[["orientacion", "orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]]
print(result)


     orientacion  orientacion_norte  orientacion_sur  orientacion_este  \
90         norte               1.00              NaN               NaN   
171     sur_este                NaN              NaN               NaN   
224         este                NaN              NaN              1.00   
304     sur_este                NaN              NaN               NaN   
354        oeste                NaN              NaN               NaN   
...          ...                ...              ...               ...   
9530         NaN                NaN             0.00              0.00   
9531         NaN                NaN             0.00              0.00   
9532         NaN                NaN             0.00              0.00   
9533         NaN                NaN             0.00              0.00   
9534         NaN                NaN             0.00              0.00   

      orientacion_oeste  
90                  NaN  
171                 NaN  
224                 NaN  
304    

In [416]:
#eliminacion de las columnas orientacion,orientacion_sur,_este,orientacion_norte,_sur
df17.drop(columns=['orientacion'], inplace=True)
df17.drop(columns=['orientacion_sur,_este'], inplace=True)
df17.drop(columns=['orientacion_norte,_sur'], inplace=True)


In [417]:
df17.query("orientacion_norte!=1 and orientacion_sur!=1  and orientacion_este!=1  and orientacion_oeste!=1 ").count()

title                 8925
url                   8925
precio                8925
precio_anterior         54
descuento               54
                      ... 
orientacion_sur        277
sistema_alarma         277
n.visitas              881
grupo_cocina          8925
clasificacion_aire    8925
Length: 106, dtype: int64

In [418]:
# Poner NaN en las columnas si no cumplen con la condición
df17.loc[df17.query("orientacion_norte!=1 and orientacion_sur!=1 and orientacion_este!=1 and orientacion_oeste!=1").index, ["orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]] = np.nan

# Asegurarse de que si alguna columna tiene valor 1, las demás sean 0
df17[["orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]] = df17[["orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]].fillna(0)

## web

In [419]:
df17["web"].value_counts(dropna=False)

web
pisos.com      4728
Indomio.com    3339
redpiso        1468
pisosmadrid     892
Name: count, dtype: int64

In [420]:
df17["fecha_descarga"].value_counts(dropna=False)

fecha_descarga
2024-11-20    3058
2024-10-19    2951
2024-08-19    1777
31/10/2024    1468
24/10/2024     892
NaN            281
Name: count, dtype: int64

In [421]:
# Sustituir NaN en la columna 'fecha_descarga' por '6 de enero de 2025'
df17["fecha_descarga"].fillna("06/01/2025", inplace=True)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7936\2424142163.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df17["fecha_descarga"].fillna("06/01/2025", inplace=True)


## certificado_energetico

In [422]:
df17["certificado_energetico"].value_counts(dropna=False)

certificado_energetico
NaN                       4286
No indicado               2134
Disponible                1986
En trámite                1492
Pendiente de completar     526
Exento                       3
Name: count, dtype: int64

In [423]:
df17.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

In [424]:
df17.query("consumoce_ano.notnull() | letrace.notnull() | emisiones_co2.notnull()" )[ ["certificado_energetico",'consumoce_ano','letrace','emisiones_co2']]

,certificado_energetico,consumoce_ano,letrace,emisiones_co2
0,Disponible,"≥ 3,51",F,D
1,No indicado,desconocido,NaN,No indicado
2,No indicado,desconocido,NaN,No indicado
3,No indicado,desconocido,NaN,No indicado
4,Disponible,desconocido,E,No indicado
...,...,...,...,...
9530,No indicado,desconocido,NaN,No indicado
9531,No indicado,desconocido,NaN,No indicado
9532,No indicado,desconocido,NaN,Inclasificable
9533,No indicado,desconocido,NaN,No indicado


In [425]:
df17["emisiones_co2"].value_counts(dropna=False)

emisiones_co2
NaN                    6827
No indicado            2142
Inclasificable          334
E                       136
D                       102
                       ... 
34.3 kg CO₂/m² añoE       1
8 kg CO₂/m² añoB          1
79 kg CO₂/m² añoF         1
45 kg CO₂/m² añoG         1
83 kg CO₂/m² añoG         1
Name: count, Length: 325, dtype: int64

In [426]:
df17["consumoce_ano"].unique()

array(['≥ 3,51', 'desconocido', '90', '258.5', '129', '284', '207', '145',
       '170', '77', '138', '120', '231.1', '224', '191', '50', '194.2',
       '216', '121', '85', '123', '399', '999', '41.2', '294', '300',
       '173', '186', '193', '400', '170.3', '172.2', '444', '176',
       '209.7', '24', '74', '164', '163', '222', '2', '328', '242', '78',
       '89', '87', '382', '342', '441', '767', '111.1', '312', '199',
       '407', '336.3', '132', '266', '55', '248', '38', '463', '188',
       '126', '414', '155', '460', '301', '44', '354', '285', '178',
       '272', '260', '324', '221', '200.8', '35', '40', '47', '158',
       '262', '130', '114.1', '212', '192', '68.5', '214', '117', '186.9',
       '245', '299', '204', '137.9', '166', '55.5', '160', '255', '189',
       '195', '263', '244', '135', '180', '110', '321', '225', '313',
       '200', '149', nan, '112', '127', '323', '213'], dtype=object)

In [427]:
df17.query("consumoce_ano=='≥ 3,51'").head()

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,acceso_discapacidad,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,barrios,cod_bario,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,consumo_energia,cal_energetica,agua_caliente,tipo_suelo,m2_utiles,gastos_comunidad,zonas_verdes,terrazas,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,cantidad_dormitorios,orientacion_este,orientacion_norte,orientacion_oeste,orientacion_sur,sistema_alarma,n.visitas,grupo_cocina,clasificacion_aire
0,"Ático Calle de Núñez de Balboa 2, Recoletos, Madrid",https://www.indomio.es/anuncios/75404850/,2000000.00,NaN,NaN,20.00,100.00,Salamanca,4.00,Recoletos,41.00,Recoletos,calle de nunez de balboa 2,Ático Calle de Núñez de Balboa 2,"≥ 3,51",F,D,Ático,1.00,Para reformar,Sólo cocina amueblada,1,1,1.00,1,1.00,1.00,2 en garaje privado,1.00,1.00,Portero media jornada,Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,1.00,1.00,0.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,1.00,-Atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test,1.00,Indomio.com,2024-11-20,Disponible,-,NaN,NaN,41.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,NaN,NaN,sin_informacion,Desconocido
11,"Piso de tres habitaciones buen estado, planta baja, San Andrés, Madrid",https://www.indomio.es/anuncios/83875795/,165000.00,NaN,NaN,2.20,75.00,Villaverde,17.00,Villaverde Alto,171,"villaverdealto,cascohistoricodevillaverde",NaN,"Piso de tres habitaciones buen estado, planta baja","≥ 3,51",F,Pendiente,Piso,1.00,Bueno / Habitable,Sí,1,3,3.00,1,1.00,1.00,NaN,1.00,0.00,NaN,"Individual, por radiadores, de gas",1971.0,4.00,"Individual, frío",Sin jardín,NaN,1.00,Libre,NaN,NaN,medio,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,Cocina independiente,1.00,0.00,1.00,1.00,0.00,0.00,NaN,1.00,0.00,0.00,0.00,0.00,"Piso en venta en edificación de buen estado pasada la ITE, planta baja.cuenta con 3 habitaciones amplias,un salón de 15m2un comedor independiente de 10m2 con vista exterioruna cocina amplia necesidad de modernizar,un amplio bañouna terraza de 20m2 que es de uso exclusivoun trastero tamaño para hacer otra habitación independiente.el piso en total de uso útil es de 90m2.potencial para crecer y hacer otros espacios.Todo esta saneado sin ninguna clase de cargas.",1.00,Indomio.com,2024-11-20,Disponible,-,0.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,NaN,NaN,independiente,Normal
12,"Piso de cuatro habitaciones Calle de Villaviciosa 4, Campamento, Madrid",https://www.indomio.es/anuncios/84201619/,240000.00,NaN,NaN,1.07,225.00,Latina,10.00,Campamento,105.00,Campamento,calle de villaviciosa 4,Piso de cuatro habitaciones Calle de Villaviciosa 4,"≥ 3,51",G,Exento,Piso,1.00,Bueno 

In [428]:
df17["letrace"].value_counts(dropna=False)

letrace
NaN    9037
E       606
D       259
F       179
G       144
C       111
A        51
B        40
Name: count, dtype: int64

#se encuentra un piso que han modificado los valores en la web
# debe tener https://www.indomio.es/anuncios/97003125/, Consumo de energia:57.9 kWh/m² año, letrace:B, e Emisiones de CO₂:11.08 kg CO₂/m² año B

In [429]:
# Asignar valores específicos si la URL coincide
for index, value in df17["url"].items():
    if value == "https://www.indomio.es/anuncios/97003125/":
        df17.at[index, "consumoce_ano"] = 57.9
        df17.at[index, "letrace"] = "B"
        df17.at[index, "emisiones_co2"] = "11.08 kg CO₂/m² año B"


revisar el codigo que trae el fichero scrapingindomio.csv , C:\Mas\Dataset_Proyecto\Ficheros_Ventas\Indomio del 12/12/2024 quiza estamos mirando solo mayusculas

#

In [ ]:
df17.query("consumoce_ano=='≥ 3,51'")[['url','letrace','emisiones_co2','fecha_descarga']]

CONSUMO Y LETRA
A: Menos de 50 kWh/m²/año
B: 51-90 kWh/m²/año
C: 91-150 kWh/m²/año
D: 151-230 kWh/m²/año
E: 231-330 kWh/m²/año
F: 331-450 kWh/m²/año
G: Más de 450 kWh/m²/año

En el caso de las emisiones:
A: Menos de 10 kg CO2/m²/año
B: 10-20 kg CO2/m²/año
C: 20-35 kg CO2/m²/año
D: 35-55 kg CO2/m²/año
E: 55-75 kg CO2/m²/año
F: 75-100 kg CO2/m²/año
G: Más de 100 kg CO2/m²/año

In [431]:
df17.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio',
       ...
       'tipo_fachada', 'cantidad_dormitorios', 'orientacion_este',
       'orientacion_norte', 'orientacion_oeste', 'orientacion_sur',
       'sistema_alarma', 'n.visitas', 'grupo_cocina', 'clasificacion_aire'],
      dtype='object', length=106)

In [432]:
#renombrar las columnas que hay de orientacion para integranlas con orientacion y sus valores

In [433]:
#Revisar que filas no tiene descripcion_extendida , deberian tenerlas todas
df17.query("descripcion_extendida.isnull()")['web'].unique()

array(['Indomio.com', 'pisos.com', 'redpiso'], dtype=object)

In [434]:

df17.query("descripcion_extendida.notnull()")['web'].unique()

array(['Indomio.com', 'redpiso', 'pisosmadrid'], dtype=object)

## dormitorios

In [435]:
df17.query("cantidad_dormitorios.isnull()")[['url','cantidad_dormitorios','dormitorios']]

,url,cantidad_dormitorios,dormitorios
0,https://www.indomio.es/anuncios/75404850/,NaN,1.00
1,https://www.indomio.es/anuncios/79178265/,NaN,2.00
2,https://www.indomio.es/anuncios/81008864/,NaN,0.00
3,https://www.indomio.es/anuncios/82164729/,NaN,1.00
4,https://www.indomio.es/anuncios/82164739/,NaN,1.00
...,...,...,...
10422,https://www.pisosmadrid.com.es/propiedad/4086493-piso-en-venta-en-madrid,NaN,2.00
10423,https://www.pisosmadrid.com.es/propiedad/4086495-piso-en-venta-en-los-angeles-madrid,NaN,3.00
10424,https://www.pisosmadrid.com.es/propiedad/4086503-piso-en-venta-en-arganzuela-madrid,NaN,4.00
10425,https://www.pisosmadrid.com.es/propiedad/4086544-piso-en-venta-en-madrid,NaN,3.00


In [436]:
df17['zonas_verdes'].value_counts(dropna=False)

zonas_verdes
NaN     8959
0.00    1337
1.00     131
Name: count, dtype: int64

In [437]:
porcentaje_nulos(df17,['zonas_verdes','reservado','nombre_oficina'])

{'zonas_verdes': 85.9211662031265,
 'reservado': 85.9211662031265,
 'nombre_oficina': 85.9211662031265}

In [438]:
df17.query("dormitorios.isnull() & cantidad_dormitorios.notnull()")[['url','dormitorios','cantidad_dormitorios']]

,url,dormitorios,cantidad_dormitorios


In [439]:
df17.drop(columns=['cantidad_dormitorios'], inplace=True)

In [440]:
df17.drop(columns=['barrios'], inplace=True)
df17.drop(columns=['cod_bario'], inplace=True)


In [441]:
df17.drop(columns=['n.visitas'], inplace=True)
df17.drop(columns=['terrazas'], inplace=True)

In [442]:
df17.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

In [ ]:
df17.columns.to_list()

In [ ]:
df17.query("consumoce_ano.notnull() or letrace.notnull() or emisiones_co2.notnull()")[['url','consumoce_ano','letrace','emisiones_co2','certificado_energetico','cal_energetica','consumo_energia']]

In [ ]:
df17.query(" cal_energetica.notnull() and consumo_energia.notnull() ")[['url','consumoce_ano','letrace','emisiones_co2','cal_energetica','consumo_energia']]

In [446]:
df17["consumo_energia"].value_counts(dropna=False)

consumo_energia
NaN                        10167
Consumo 346 KW hm2 ano        14
Consumo 222 KW hm2 ano         5
Consumo 257 KW hm2 ano         4
Consumo 204 KW hm2 ano         4
                           ...  
Consumo 309 KW hm2 ano         1
Consumo 130 KW hm2 ano         1
Consumo 261 KW hm2 ano         1
Consumo 1530 KW hm2 ano        1
Consumo 417 KW hm2 ano         1
Name: count, Length: 171, dtype: int64

In [447]:
# Función para extraer solo el número de la cadena de texto
def extraer_numero(texto):
    if isinstance(texto, str):
        match = re.search(r'\d+', texto)
        return match.group(0) if match else None
    return None

In [448]:
df17.loc[df17['consumoce_ano'].isnull(), 'consumoce_ano'] = df17.loc[df17['consumoce_ano'].isnull(), 'consumo_energia'].apply(lambda x: extraer_numero(x) if isinstance(x, str) else None)

In [449]:
columns_to_replace = ['consumoce_ano', 'letrace', 'emisiones_co2','cal_energetica']
df17[columns_to_replace] = df17[columns_to_replace].replace(['desconocido', 'No indicado', 'Inclasificable','Pendiente','Exento','Cal energetica','Cal energetica Exento '], np.nan)

In [ ]:
df17.query("consumo_energia.notnull() ")[['url','consumoce_ano','letrace','emisiones_co2','consumo_energia','emisiones_co2','cal_energetica']]

In [451]:
df17["emisiones_co2"].value_counts(dropna=False)

emisiones_co2
NaN                    9330
E                       136
D                       102
F                        81
C                        53
                       ... 
34.3 kg CO₂/m² añoE       1
8 kg CO₂/m² añoB          1
79 kg CO₂/m² añoF         1
45 kg CO₂/m² añoG         1
83 kg CO₂/m² añoG         1
Name: count, Length: 321, dtype: int64

In [452]:
df17['emisiones_co2'] = df17['emisiones_co2'].str.replace('kg CO₂/m² año', '')

In [453]:
# Reemplazar 'NaN' con valores nulos de Pandas
df17['emisiones_co2'] = df17['emisiones_co2'].replace('NaN', np.nan)

# Extraer letra_emisiones y emisiones_ano
df17[['emisiones_ano', 'letra_emisiones']] = df17['emisiones_co2'].str.extract(r'(?:(\d+\.\d+|\d+)\s)?([A-Z])')

# Convertir emisiones_ano a float
df17['emisiones_ano'] = df17['emisiones_ano'].astype(float)

In [454]:
df17['emisiones_ano'].value_counts(dropna=False)

emisiones_ano
NaN      10043
44.00       11
42.00        9
56.00        8
36.00        8
         ...  
49.80        1
58.10        1
18.00        1
36.50        1
83.00        1
Name: count, Length: 181, dtype: int64

In [455]:
df17.query("consumoce_ano.notnull() and letrace.isnull()")[['url','consumoce_ano','letrace','emisiones_co2']]

,url,consumoce_ano,letrace,emisiones_co2
7788,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126007,111,NaN,Emisiones 18 Kg CO2m2 ano
7789,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126427,145,NaN,Emisiones 24 Kg CO2m2 ano
7792,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126442,104,NaN,Emisiones 17 Kg CO2m2 ano
7798,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126423,88,NaN,Emisiones 14 Kg CO2m2 ano
7799,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-alejandro-sanchez-opanel-carabanchel-madrid-madrid-RP512023115395,600,NaN,Emisiones 149 Kg CO2m2 ano
...,...,...,...,...
9211,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-fresnedillas-fuentelarreina-fuencarral-el-pardo-madrid-madrid-RP722023118677,198,NaN,Emisiones 41 Kg CO2m2 ano
9216,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-general-pardinas-58-lista-salamanca-madrid-madrid-RP1432024124685,326,NaN,Emisiones 68 Kg CO2m2 ano
9222,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-orense-cuatro-caminos-tetuan-madrid-madrid-RP2332023114339,138,NaN,Emisiones 34 Kg CO2m2 ano
9241,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-casado-del-alisal-jeronimos-retiro-madrid-madrid-RP1792023115851,180,NaN,Emisiones 45 Kg CO2m2 ano


In [456]:
df17.query("consumo_energia.notnull() or letrace.notnull() or emisiones_co2.notnull()")[['url','consumoce_ano','letrace','emisiones_co2','letra_emisiones','emisiones_ano']]

,url,consumoce_ano,letrace,emisiones_co2,letra_emisiones,emisiones_ano
0,https://www.indomio.es/anuncios/75404850/,"≥ 3,51",F,D,D,NaN
4,https://www.indomio.es/anuncios/82164739/,NaN,E,NaN,NaN,NaN
11,https://www.indomio.es/anuncios/83875795/,"≥ 3,51",F,NaN,NaN,NaN
12,https://www.indomio.es/anuncios/84201619/,"≥ 3,51",G,NaN,NaN,NaN
15,https://www.indomio.es/anuncios/87473239/,NaN,D,D,D,NaN
...,...,...,...,...,...,...
9520,https://www.indomio.es/anuncios/96979909/,NaN,C,C,C,NaN
9522,https://www.indomio.es/anuncios/96696025/,NaN,C,B,B,NaN
9524,https://www.indomio.es/anuncios/96950227/,NaN,G,NaN,NaN,NaN
9527,https://www.indomio.es/anuncios/97011087/,NaN,E,NaN,NaN,NaN


In [457]:
pd.set_option('display.max_rows', None)

In [458]:
#eliminar las columnas consumo_energia,cal_energetica

In [459]:
df17 = df17.drop(columns=['consumo_energia', 'cal_energetica'])

In [460]:
df17.head(1)

,title,url,precio,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,detalle,consumoce_ano,letrace,emisiones_co2,tipologia,propiedad_completa,estado,amueblado,planta,habitaciones,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,porteria,calefaccion,ano_construccion,plantas_edificio,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,disponibilidad,nuda_propiedad,sup_comercial,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,instalacion_tv_centralizada,instalacion_tv_individual,exterior,interior,interior_y_exterior,tiene_piscina,porton_electrico,puerta_blindada,alarma,videoportero,lujosa,descripcion_extendida,acceso_discapacidad,web,fecha_descarga,certificado_energetico,piscina,tiene_jardin,referencia,cantidad_visitas,fecha_publicacion,titular_anuncio,campo12,cantidad_armarios,agua_caliente,tipo_suelo,m2_utiles,gastos_comunidad,zonas_verdes,puerta_seguridad,num_plantas,num_pisos,reservado,nombre_oficina,cod_barrio_2,codigo_Postal,tipo_fachada,orientacion_este,orientacion_norte,orientacion_oeste,orientacion_sur,sistema_alarma,grupo_cocina,clasificacion_aire,emisiones_ano,letra_emisiones
0,"Ático Calle de Núñez de Balboa 2, Recoletos, Madrid",https://www.indomio.es/anuncios/75404850/,2000000.00,NaN,NaN,20.00,100.00,Salamanca,4.00,Recoletos,41.00,Recoletos,calle de nunez de balboa 2,Ático Calle de Núñez de Balboa 2,"≥ 3,51",F,D,Ático,1.00,Para reformar,Sólo cocina amueblada,1,1,1.00,1,1.00,1.00,2 en garaje privado,1.00,1.00,Portero media jornada,Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,1.00,1.00,0.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,1.00,-Atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test,1.00,Indomio.com,2024-11-20,Disponible,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,NaN,sin_informacion,Desconocido,NaN,D


In [461]:
# df17[['consumo_energia', 'cal_energetica','letrace','letra_emisiones']]

In [462]:
# df17.query("certificado_energetico.notnull() and certificado_energetico!='No indicado'")[['consumo_energia', 'consumoce_ano','emisiones_ano','letrace','letra_emisiones','certificado_energetico']]

In [463]:
# df17.query("certificado_energetico.notnull() and certificado_energetico!='No indicado'")[['consumo_energia', 'consumoce_ano','emisiones_ano','letrace','letra_emisiones','certificado_energetico']]

In [464]:
df17.to_csv("pisoscompleto.csv",index='False')

# Lectura pisoscompleto

In [3]:
df17 = pd.read_csv("pisoscompleto.csv", index_col=False)

C:\Users\extas\AppData\Local\Temp\ipykernel_20028\4238198241.py:1: DtypeWarning: Columns (3,5,14,26,30,72,76,83) have mixed types. Specify dtype option on import or set low_memory=False.
  df17 = pd.read_csv("pisoscompleto.csv", index_col=False)


In [19]:


# Palabras clave para clasificar
central_keywords = ["central", "comunitaria", "comunitario"]
individual_keywords = ["individual", "particular", "chimenea", "bomba", "termo", "climatizacion"]
electric_keywords = ["electrica", "bomba", "climatización", 'termo electrico','climatizacion']
gas_keywords = ["gas", "propano", "g n", "g/n"]  # Asegurarse de que 'g n' y 'g/n' estén bien escritos
gasoil_keywords = ["gasóil","gasoil",'gasoil',"Gasoil"]  # Añadir "gasoil" aquí
renovables_keywords = ["aerotermia", "geotermia", "chimenea"]
suelo_radiante_keywords=["suelo_radiante", "suelo"]
#radiadores_keywords=["radiadores"]
aire_keywords=["bomba","climatización",'climatizacion','conducto']
desconocido_keywords=['chimenea', 'termo electrico']
#lo mas extendido son los radiadores por eso despues de pasar , suelo radiante y aire pongo a todos radiadores salvo a chimenea y termo electrico que no lo se
radiadores_keywords=['radiadores','Gas natural', 'G/n', 'Gas particular', 'Central, alimentación eléctrica', 'Agua caliente y calefacción central con contador i', 'Comunitario', 'Individual con gas natural', 'Individual, de gasóil', 'Comunitaria', 'Calefaccion Propano', 'Calefaccion Gas Natural', 'Central, por aire, de gas', 'Gas natural y biomasa', 'Centralizada', 'Eléctrica', 'Calefaccion Individual', 'Calefaccion individual','Contadores individuales', '1.0', 'Individual, alimentación eléctrica', 'Aerotermia', 'Individual - gas natural', 'Individual', 'Calefaccion Aerotermia', 'Gasoil', 'Central individualizada', 'Individual, por aire, alimentación eléctrica', 'Geotermia y aerotermia', 'Calefaccion comunitaria', 'Calefaccion Central', 'Central, de gas', 'Calefaccion Electrica', 'Calefacción central individualizada', 'Central, de gasóil', 'Individual, de gas', 'Central']

# Inicializar conjuntos para valores únicos
central = set()
individual = set()
electrica = set()
gas = set()
gsoil = set()
renovables = set()
suelo_radiante = set()
radiadores = set()
aire = set()
sin_clasificar = set()
sin_clasificar_energia = set()
sin_clasificar_medio = set()
radiadores=set()
suelo_radiante=set()


# Clasificar categorías
for categoria in df17["calefaccion"]:
    categoria_normalized = normalize_text(str(categoria))  # Normalizar texto
    
 
    
    # Clasificación por tipo (central o individual)
    if any(keyword in categoria_normalized for keyword in central_keywords):
        central.add(categoria)
    elif any(keyword in categoria_normalized for keyword in individual_keywords):
        individual.add(categoria)
    else:
        sin_clasificar.add(categoria)
    
    # Clasificación por energía (eléctrica, gas, gasoil)
    if any(keyword in categoria_normalized for keyword in electric_keywords):
        electrica.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gasoil_keywords):
        gsoil.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gas_keywords):
        gas.add(categoria)
      
   
    elif any(keyword in categoria_normalized for keyword in renovables_keywords):
        renovables.add(categoria)
    else:
        sin_clasificar_energia.add(categoria)

    # Clasificación por medio (suelo radiante, radiadores, aire)
    
   
    if any(keyword in categoria_normalized for keyword in aire_keywords):
        aire.add(categoria)    
    elif any(keyword in categoria_normalized for keyword in suelo_radiante_keywords):
        suelo_radiante.add(categoria)
    elif any(keyword in categoria_normalized for keyword in desconocido_keywords):
        sin_clasificar_medio.add(categoria)
  #  else: any(keyword in categoria_normalized for keyword in radiadores_keywords):
    else: 
        radiadores.add(categoria)
    

# Convertir los conjuntos a listas
central = list(central)
individual = list(individual)
sin_clasificar = list(sin_clasificar)
electrica = list(electrica)
gas = list(gas)
gasoil = list(gsoil)
renovables = list(renovables)
sin_clasificar_energia = list(sin_clasificar_energia)
aire=list(aire)
radiadores=list(radiadores)
suelo_radiante=list(suelo_radiante)
sin_clasificar_medio=list(sin_clasificar_medio)
# Imprimir resultados
print("Central:", central)
print("Individual:", individual)
print("No clasificado:", sin_clasificar)
print("Electrica:", electrica)
print("Gas:", gas)
print("Gasoil:", gasoil)
print("Renovables:", renovables)
print("No clasificado en energía:", sin_clasificar_energia)
print("Suelo Radiante:", suelo_radiante)
print("Radiadores:", radiadores)
print("Aire:", aire)
print("No clasificado en medio:", sin_clasificar_medio)


Central: ['Calefaccion comunitaria', 'Agua caliente y calefacción central con contador i', 'Central', 'Calefaccion Central', 'Calefacción central individualizada', 'Central, por radiadores, de gas', 'Comunitario', 'Central, por aire, de gas', 'Central, por radiadores', 'Comunitaria', 'Central individualizada', 'Central, de gas', 'Central, de gasóil', 'Centralizada', 'Central, por radiadores, de gasóil', 'Central, suelo radiante, de gas', 'Central, suelo radiante', 'Central, alimentación eléctrica']
Individual: ['Gas particular', 'Climatizacion', 'Individual, suelo radiante', 'Calefaccion individual', 'Contadores individuales', 'climatizacion', 'Individual, por radiadores', 'Individual, por radiadores, de gasóil', 'Chimenea', 'Individual, alimentación eléctrica', 'Bomba frio-calor', 'Termo electrico', 'Individual, por aire, por bomba de calor', 'Individual, por radiadores, de gas', 'Individual con gas natural', 'Individual, suelo radiante, de gas', 'Bomba de calor', 'Individual, por air

In [20]:

# Función para clasificar el tipo de calefacción y asignar el nombre de la lista
def clasificar_calefaccion(calefaccion, listas):
    for tipo, lista in listas.items():
        if calefaccion in lista:
            return tipo
    return 'No clasif'




In [22]:
# Listas proporcionadas
listas = {
    'Central': ['Calefaccion comunitaria', 'Agua caliente y calefacción central con contador i', 'Central', 'Calefaccion Central', 'Calefacción central individualizada', 'Central, por radiadores, de gas', 'Comunitario', 'Central, por aire, de gas', 'Central, por radiadores', 'Comunitaria', 'Central individualizada', 'Central, de gas', 'Central, de gasóil', 'Centralizada', 'Central, por radiadores, de gasóil', 'Central, suelo radiante, de gas', 'Central, suelo radiante', 'Central, alimentación eléctrica'],
    'Individual': ['Gas particular', 'Climatizacion', 'Individual, suelo radiante', 'Calefaccion individual', 'Contadores individuales', 'climatizacion', 'Individual, por radiadores', 'Individual, por radiadores, de gasóil', 'Chimenea', 'Individual, alimentación eléctrica', 'Bomba frio-calor', 'Termo electrico', 'Individual, por aire, por bomba de calor', 'Individual, por radiadores, de gas', 'Individual con gas natural', 'Individual, suelo radiante, de gas', 'Bomba de calor', 'Individual, por aire, alimentación eléctrica', 'Bomba frio/calor', 'Individual - gas natural', 'Bomba frio calor', 'Individual, de gas', 'Calefaccion Individual', 'Individual, por radiadores, alimentación eléctrica', 'Por climatizacion', 'Individual', 'Individual, de gasóil'],
    'No clasificado': ['Calefaccion Propano', 'Calefaccion Electrica', 'Eléctrica', 'Suelo radiante agua', 'Conducto', 'Gasoil', 'Calefaccion Gas Natural', 'G/n', 'Geotermia y aerotermia', 'Calefaccion Aerotermia', '1.0', 'Calefaccion Suelo Radiante', 'Aerotermia', np.nan, 'Gas natural', 'Gas natural y biomasa', 'Suelo radiante'],
    'Electrica': ['Individual, por aire, por bomba de calor', 'Individual, por aire, alimentación eléctrica', 'Bomba frio calor', 'Calefaccion Electrica', 'Individual, por radiadores, alimentación eléctrica', 'Climatizacion', 'Eléctrica', 'Bomba frio/calor', 'Por climatizacion', 'climatizacion', 'Individual, alimentación eléctrica', 'Bomba de calor', 'Bomba frio-calor', 'Termo electrico', 'Central, alimentación eléctrica'],
    'Gas': ['Calefaccion Propano', 'Gas particular', 'Individual, por radiadores, de gas', 'Individual, de gas', 'Central, por radiadores, de gas', 'Individual con gas natural', 'Individual, suelo radiante, de gas', 'Central, por aire, de gas', 'Calefaccion Gas Natural', 'G/n', 'Individual - gas natural', 'Central, de gas', 'Central, suelo radiante, de gas', 'Gas natural', 'Gas natural y biomasa'],
    'Gasoil': ['Gasoil', 'Individual, por radiadores, de gasóil', 'Central, de gasóil', 'Central, por radiadores, de gasóil', 'Individual, de gasóil'],
    'Renovables': ['Calefaccion Aerotermia', 'Chimenea', 'Aerotermia', 'Geotermia y aerotermia'],
    'No clasificado en energía': ['Comunitario', 'Individual, suelo radiante', 'Calefaccion individual', 'Contadores individuales', 'Individual, por radiadores', 'Centralizada', 'Suelo radiante', 'Calefaccion Central', 'Calefacción central individualizada', 'Conducto', 'Central, suelo radiante', 'Agua caliente y calefacción central con contador i', 'Comunitaria', 'Central individualizada', 'Calefaccion Suelo Radiante', np.nan, 'Central, por radiadores', 'Calefaccion comunitaria', 'Central', 'Calefaccion Individual', 'Suelo radiante agua', '1.0', 'Individual'],
    'Suelo Radiante': ['Individual, suelo radiante, de gas', 'Suelo radiante agua', 'Individual, suelo radiante', 'Calefaccion Suelo Radiante', 'Central, suelo radiante, de gas', 'Central, suelo radiante', 'Suelo radiante'],
    'Radiadores': ['Gas particular', 'Calefaccion Electrica', 'Central, por radiadores, de gas', 'Comunitario', 'Central, por aire, de gas', 'Eléctrica', 'Gasoil', 'Geotermia y aerotermia', 'Calefaccion individual', 'Contadores individuales', 'Calefaccion Aerotermia', 'Individual, por radiadores', 'Individual, por radiadores, de gasóil', 'Centralizada', 'Individual, alimentación eléctrica', 'Calefaccion Central', 'Individual, por radiadores, de gas', 'Calefacción central individualizada', 'Individual con gas natural', 'G/n', 'Central, de gas', 'Central, de gasóil', 'Central, por radiadores, de gasóil', 'Agua caliente y calefacción central con contador i', 'Calefaccion Propano', 'Individual, por aire, alimentación eléctrica', 'Calefaccion Gas Natural', 'Comunitaria', 'Individual - gas natural', 'Central individualizada', np.nan, 'Central, por radiadores', 'Central, alimentación eléctrica', 'Calefaccion comunitaria', 'Central', 'Individual, de gas', 'Calefaccion Individual', 'Individual, por radiadores, alimentación eléctrica', '1.0', 'Individual', 'Aerotermia', 'Gas natural', 'Gas natural y biomasa', 'Individual, de gasóil'],
    'Aire': ['Individual, por aire, por bomba de calor', 'Bomba frio calor', 'Climatizacion', 'Conducto', 'Bomba frio/calor', 'Por climatizacion', 'climatizacion', 'Bomba de calor', 'Bomba frio-calor'],
    'No clasificado en medio': ['Termo electrico', 'Chimenea']
}

In [23]:
# Definir las listas específicas para cada columna nueva
listas_tipo_1 = {k: listas[k] for k in ['Central', 'Individual']}
listas_tipo_2 = {k: listas[k] for k in ['Electrica', 'Gas', 'Gasoil','Renovables','No clasificado en energía']}
listas_tipo_3 = {k: listas[k] for k in ['Suelo Radiante', 'Radiadores', 'Aire', 'No clasificado en medio']}

# Aplicar la función a la columna calefaccion y crear tres nuevas columnas
df17['tipo_calefaccion'] = df17['calefaccion'].apply(lambda x: clasificar_calefaccion(x, listas_tipo_1))
df17['energia_calefaccion'] = df17['calefaccion'].apply(lambda x: clasificar_calefaccion(x, listas_tipo_2))
df17['instalacion_calefaccion'] = df17['calefaccion'].apply(lambda x: clasificar_calefaccion(x, listas_tipo_3))



In [32]:
df17["amueblado"].value_counts(dropna=False)

amueblado
NaN                       7531
0                         1467
0.0                        720
Sí                         293
No                         171
1.0                        161
Sólo cocina amueblada       71
Parcialmente amueblado      12
1                            1
Name: count, dtype: int64

In [ ]:
df17.columns.tolist()

In [33]:
df17.isna().sum()

Unnamed: 0                     0
title                          0
url                            0
precio                         0
precio_anterior            10373
                           ...  
emisiones_ano              10043
letra_emisiones             9330
tipo_calefaccion               0
energia_calefaccion            0
instalacion_calefaccion        0
Length: 105, dtype: int64

In [471]:
df17.isna().mean()*100

precio                                       0.00
m2_constr                                    0.26
cod_distrito                                 0.02
cod_barrio                                   2.28
consumoce_ano                               95.20
letrace                                     86.67
tipologia                                    0.00
estado                                      45.63
amueblado                                   72.23
planta                                      44.53
dormitorios                                  1.82
banos                                        2.92
garaje                                      80.29
trastero                                    16.30
ascensor                                    22.14
porteria                                    98.54
calefaccion                                 66.07
ano_construccion                            66.58
aire_acondicionado                          56.78
jardin                                      79.02


In [472]:
# Chequeo de valores en columna 
df17["sistema_alarma"].value_counts()

sistema_alarma
0.00    276
1.00      5
Name: count, dtype: int64

In [473]:
df17["orientacion_sur"].value_counts()

orientacion_sur
0.00    9943
1.00     484
Name: count, dtype: int64

In [474]:
df17["orientacion_oeste"].value_counts()

orientacion_oeste
0.00    10048
1.00      379
Name: count, dtype: int64

In [475]:
df17["orientacion_norte"].value_counts()

orientacion_norte
0.00    10108
1.00      319
Name: count, dtype: int64

In [476]:
df17["orientacion_este"].value_counts()

orientacion_este
0.00    10003
1.00      424
Name: count, dtype: int64

In [477]:
df17["puerta_seguridad"].value_counts()

puerta_seguridad
0.00    1450
1.00      18
Name: count, dtype: int64

In [478]:
df17["zonas_verdes"].value_counts()

zonas_verdes
0.00    1337
1.00     131
Name: count, dtype: int64

In [479]:
df17["codigo_Postal"].unique()

array([   nan, 28021., 28041., 28025., 28029., 28037., 28019., 28045.,
       28018., 28017., 28044., 28015., 28010., 28027., 28030., 28011.,
       28024., 28033., 28007., 28047., 28032., 28028., 28034., 28031.,
       28016., 28002., 28012., 28005., 28003., 28001., 28004., 28039.,
       28038., 28026., 28053., 28020., 28035., 28055., 28046., 28023.,
       28009., 28042., 28006., 28043., 28048., 28049., 28013., 28014.,
       28040., 28008.])

In [76]:
df17.to_csv("pisoscompleto.csv",index='False')

# Lectura pisoscompleto

In [17]:
df17 = pd.read_csv("pisoscompleto.csv", index_col=False)

C:\Users\extas\AppData\Local\Temp\ipykernel_11656\4238198241.py:1: DtypeWarning: Columns (5,7,16,28,32,74,78,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df17 = pd.read_csv("pisoscompleto.csv", index_col=False)


In [25]:
dfadicionales= pd.read_csv('../02-datos-limpios/Datos_adicionales.csv',sep=";", index_col=False)

In [19]:
dfadicionales.columns

Index(['cod_municipio', 'municipio', 'cod_distrito', 'distrito', 'cod_barrio',
       'barrio', 'codigo_Postal', 'num_personas', 'num_personas_hombres',
       'num_personas_mujeres', 'total_transacciones',
       'viv_nuevas_transacciones', 'viv_usadas_transacciones', 'lineas_metro',
       'salidas_metro', 'estaciones_metroligero', 'estaciones_cercanias',
       'estacion_bus_urbano', 'num_lineas_buses',
       'Renta neta media por persona', 'Renta neta media por hogar',
       'Mediana de la renta por unidad de consumo',
       'Renta bruta media por persona', 'Renta bruta media por hogar',
       'Habitantes', 'Hogares', 'Tamaño medio del hogar',
       'Tamaño del hogar (Nº de personas en la vivienda)', 'hog_con_1',
       'hog_con_2', 'hog_con_3', 'hog_con_4', 'hog_con_5', 'hog_con_6',
       'hog_con_7', 'hog_con_8', 'hog_con_9', 'hog_con_10', 'hog_con_11',
       'hog_con_12', 'hog_con_13', 'hog_con_14', 'hog_con_15 y más'],
      dtype='object')

In [ ]:
#uno los datos adicionales a nivel de barrio para las columnas  codigo_Postal,	num_personas,	num_personas_hombres	,num_personas_mujeres	,total_transacciones,	viv_nuevas_transacciones,	viv_usadas_transacciones
# uno a nivel de distrito lineas_metro	salidas_metro	estaciones_metroligero	estaciones_cercanias	estacion_bus_urbano	num_lineas_buses	Renta neta media por persona	Renta neta media por hogar	Mediana de la renta por unidad de consumo	Renta bruta media por persona	Renta bruta media por hogar	Habitantes	Hogares	Tamaño medio del hogar	Tamaño del hogar (Nº de personas en la vivienda)	hog_con_1	hog_con_2	hog_con_3	hog_con_4	hog_con_5	hog_con_6	hog_con_7	hog_con_8	hog_con_9	hog_con_10	hog_con_11	hog_con_12	hog_con_13	hog_con_14	hog_con_15 y más


In [ ]:
df17.query("cod_barrio<11")

In [20]:
df17.loc[df17['url'].str.contains('calle-narvaez'), ['cod_barrio', 'barrio']] = [34, 'Ibiza']
df17.loc[df17['url'].str.contains('calle-claudio-coello'), ['cod_barrio', 'barrio']] = [41, 'Recoletos']
df17.loc[df17['url'].str.contains('calle-maria-de-molina'), ['cod_barrio', 'barrio']] = [51, 'El Viso']
df17.loc[df17['url'].str.contains('calle-eloy-gonzalo'), ['cod_barrio', 'barrio']] = [73, 'Trafalgar']

df17['titular_anuncio'] = df17['descripcion_extendida'].fillna('')
df17.loc[df17['titular_anuncio'].str.contains('Ibiza'), ['cod_barrio', 'barrio']] = [34, 'Ibiza']


# Reemplazar valores NaN en 'descripcion_extendida' con una cadena vacía
df17['descripcion_extendida'] = df17['descripcion_extendida'].fillna('')
df17.loc[df17['descripcion_extendida'].str.contains('Calle Mariano Benlliure'), ['cod_barrio', 'barrio','cod_distrito','distrito']] = [74, 'Almagro',7,'Chamberí']
df17.loc[df17['descripcion_extendida'].str.contains('Calle Moquetas 11'), ['cod_barrio', 'barrio','cod_distrito','distrito','estado']] = [74, 'Almagro',7,'Chamberí','Obra nueva']
df17.loc[df17['descripcion_extendida'].str.contains('calle de Castello'), ['cod_barrio', 'barrio']] = [74, 'Almagro']
df17.loc[df17['descripcion_extendida'].str.contains('IBIZA'), ['cod_barrio', 'barrio']] = [34, 'Ibiza']

In [23]:
df17["m2_utiles"].value_counts(dropna=False)

m2_utiles
NaN      8961
 60        45
 55        37
 90        37
 70        36
         ... 
 245        1
 194        1
 383        1
 244        1
 317        1
Name: count, Length: 191, dtype: int64

In [22]:
df17["m2_utiles"]=df17["m2_utiles"].str.replace("Metros","")

In [ ]:
df17.drop_duplicates(subset=['url'], keep='first', inplace=True)


In [ ]:
df17['url'].nunique() == len(df17)

Elimino la columna codigo porstal por que la voy a traer al hacer el merge con dfadicionales

In [38]:
df17 = df17.drop(columns="codigo_Postal")

In [39]:
df17.to_csv("pisoscompleto.csv",index='False')
df17 = pd.read_csv("pisoscompleto.csv", index_col=False)

In [24]:
df17 = pd.read_csv("pisoscompleto.csv", index_col=False)

C:\Users\extas\AppData\Local\Temp\ipykernel_17088\4238198241.py:1: DtypeWarning: Columns (7,9,18,30,34,76,80,87) have mixed types. Specify dtype option on import or set low_memory=False.
  df17 = pd.read_csv("pisoscompleto.csv", index_col=False)


In [26]:


# Seleccionar solo las columnas que quieres añadir
columns_to_add = ['cod_barrio', 'codigo_Postal', 'num_personas', 'num_personas_hombres', 'num_personas_mujeres', 'total_transacciones', 'viv_nuevas_transacciones', 'viv_usadas_transacciones']
columns_to_add_d = ['cod_distrito', 'codigo_Postal', 'num_personas', 'num_personas_hombres', 'num_personas_mujeres', 'total_transacciones', 'viv_nuevas_transacciones', 'viv_usadas_transacciones']

# Filtrar el DataFrame adicional para mantener solo las columnas seleccionadas
df_adicional_filtered = dfadicionales[columns_to_add]
#df_adicional_filtered_d = dfadicionales[columns_to_add_d]
df_adicional_filtered_d = dfadicionales[dfadicionales['cod_barrio'].isna()][columns_to_add_d]

# Filtrar filas en df17 donde cod_barrio no es nulo y hacer el primer merge
df17_filtered = df17.dropna(subset=['cod_barrio'])
df_merged_barrio = pd.merge(df17_filtered, df_adicional_filtered, on='cod_barrio', how='left')

# Filtrar filas en df17 donde cod_barrio es nulo
df17_na = df17[df17['cod_barrio'].isna()]

# Realizar el merge con las filas filtradas usando cod_distrito
df_merged_na = pd.merge(df17_na, df_adicional_filtered_d, on='cod_distrito', how='left')

# Combinar los dos DataFrames resultantes en uno final
df_final = pd.concat([df_merged_barrio, df_merged_na], ignore_index=True)




In [27]:
df_final.columns.tolist()

['Unnamed: 0.4',
 'Unnamed: 0.3',
 'Unnamed: 0.2',
 'Unnamed: 0.1',
 'Unnamed: 0',
 'title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/p

In [28]:
df17["cod_barrio"].isnull().sum()

237

In [141]:
df17.shape,df_merge.shape,df_merged_na.shape, df17_na.shape,df_adicional_filtered_d.shape,df_final.shape

((10427, 107), (10190, 113), (237, 114), (237, 107), (21, 8), (10427, 114))

In [ ]:
#ahora hago lo mismo para las que voy a hacer merge por distrito

In [29]:
#ahora añado las columnas que estan a nivel de distrito
#lineas_metro	salidas_metro	estaciones_metroligero	estaciones_cercanias	estacion_bus_urbano	num_lineas_buses	Renta neta media por persona	Renta neta media por hogar	Mediana de la renta por unidad de consumo	Renta bruta media por persona	Renta bruta media por hogar	Habitantes	Hogares	Tamaño medio del hogar	Tamaño del hogar (Nº de personas en la vivienda)	hog_con_1	hog_con_2	hog_con_3	hog_con_4	hog_con_5	hog_con_6	hog_con_7	hog_con_8	hog_con_9	hog_con_10	hog_con_11	hog_con_12	hog_con_13	hog_con_14	hog_con_15 y más
columns_to_add_d = ['cod_distrito','lineas_metro',	'salidas_metro',	'estaciones_metroligero',	'estaciones_cercanias',	'estacion_bus_urbano',	'num_lineas_buses',	'Renta neta media por persona',	'Renta neta media por hogar','Mediana de la renta por unidad de consumo','Renta bruta media por persona','Renta bruta media por hogar','Habitantes','Hogares','Tamaño medio del hogar','Tamaño del hogar (Nº de personas en la vivienda)','hog_con_1','hog_con_2','hog_con_3'	,'hog_con_4','hog_con_5','hog_con_6','hog_con_7','hog_con_8','hog_con_9','hog_con_10','hog_con_11',	'hog_con_12','hog_con_13','hog_con_14','hog_con_15 y más']

df_adicional_filtered_d = dfadicionales[dfadicionales['cod_barrio'].isna()][columns_to_add_d]

# Unir los DataFrames por 'cod_barrio'
df_final = pd.merge(df_final, df_adicional_filtered_d, on='cod_distrito', how='left')

In [10]:
df_final.shape

(10421, 145)

In [30]:
# Filtrar las filas que no se pueden convertir a float sin modificar la columna 'precio'
df_final[~df_final['precio'].astype(str).str.replace(',', '').str.replace(' ', '').str.isnumeric()][["title","url","precio"]]



,title,url,precio
0,"Ático Calle de Núñez de Balboa 2, Recoletos, Madrid",https://www.indomio.es/anuncios/75404850/,2000000.00
1,"Piso de dos habitaciones buen estado, entreplanta, Casco Histórico de Barajas, Madrid",https://www.indomio.es/anuncios/79178265/,287000.00
2,"Casa plurifamiliar Plaza de Italia, Legazpi, Madrid",https://www.indomio.es/anuncios/81008864/,200000.00
3,"Piso de una habitación Calle de Juan Ramón Jiménez 8, Hispanoamérica, Madrid",https://www.indomio.es/anuncios/82164729/,650000.00
4,"Piso de una habitación Calle de Juan Ramón Jiménez 8, Hispanoamérica, Madrid",https://www.indomio.es/anuncios/82164739/,650000.00
...,...,...,...
9524,"Piso de dos habitaciones Calle Antillon, Puerta del Ángel, Madrid",https://www.indomio.es/anuncios/96974257/,249.900
9525,"Piso de dos habitaciones 127 m², Peñagrande, Madrid",https://www.indomio.es/anuncios/96635079/,690.000
9526,"Piso de una habitación Calle Del Monasterio De Las Batuecas, Mirasierra, Madrid",https://www.indomio.es/anuncios/96931857/,375.000
9527,"Piso Calle de Joaquín Lorenzo, Peñagrande, Madrid",https://www.indomio.es/anuncios/97013237/,165.000


In [60]:


# Ver las primeras filas de la columna 'precio' para verificar el formato
print(df_final['precio'].tail(20))





10401    2475000
10402     238000
10403     239000
10404     380000
10405    1150000
10406     301000
10407     655000
10408    1770000
10409     235000
10410     850000
10411     539000
10412    1355000
10413     460000
10414     570000
10415     285000
10416     575000
10417     870000
10418     315000
10419      97920
10420     329000
Name: precio, dtype: object


In [62]:
df_final["web"].value_counts(dropna=False)

web
pisos.com      4728
Indomio.com    3334
redpiso        1467
pisosmadrid     892
Name: count, dtype: int64

In [69]:
df_final["fecha_descarga"].value_counts(dropna=False)

fecha_descarga
2024-11-20    3055
2024-10-19    2951
2024-08-19    1777
31/10/2024    1467
24/10/2024     892
06/01/2025     279
Name: count, dtype: int64

In [31]:
# Eliminar los puntos de la columna 'precio' y convertirla a float
df_final.loc[(df_final['fecha_descarga'] == '06/01/2025') & (df_final['web'] == 'Indomio.com'), 'precio'] = df_final.loc[(df_final['fecha_descarga'] == '06/01/2025') & (df_final['web'] == 'Indomio.com'), 'precio'].astype(str).apply(lambda x: x.replace('.', '')).astype(float)



In [32]:
df_final.loc[df_final['web'] == 'Indomio.com', 'precio'] = df_final.loc[df_final['web'] == 'Indomio.com', 'precio'].astype(float)

In [33]:
# Filtrar las filas donde 'web' es igual a 'redpiso' y luego convertir la columna 'precio' a float
df_final.loc[df_final['web'] == 'redpiso', 'precio'] = df_final.loc[df_final['web'] == 'redpiso', 'precio'].astype(float)
df_final.loc[df_final['web'] == 'pisosmadrid', 'precio'] = df_final.loc[df_final['web'] == 'pisosmadrid', 'precio'].astype(float)
df_final.loc[df_final['web'] == 'pisos.com', 'precio'] = df_final.loc[df_final['web'] == 'pisos.com', 'precio'].astype(float)

# Mostrar las primeras filas del dataframe actualizado
print(df_final[['web', 'precio']].sample(20))


               web     precio
7646     pisos.com  210000.00
10295  pisosmadrid  103500.00
2042   Indomio.com 2000000.00
3313     pisos.com  995000.00
5640     pisos.com  210000.00
3178     pisos.com  255000.00
41     Indomio.com 1790000.00
2254   Indomio.com  430000.00
5285     pisos.com 1750000.00
5595     pisos.com  896500.00
8896       redpiso  199000.00
6812     pisos.com 2590000.00
4580     pisos.com  785000.00
5965     pisos.com 1179000.00
5465     pisos.com 1839000.00
5577     pisos.com  950000.00
192    Indomio.com  960000.00
305    Indomio.com  366000.00
8430       redpiso  148500.00
656    Indomio.com 1339000.00


In [34]:
df_final.columns.tolist()

['Unnamed: 0.4',
 'Unnamed: 0.3',
 'Unnamed: 0.2',
 'Unnamed: 0.1',
 'Unnamed: 0',
 'title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/p

In [35]:
df_final.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2', 'Unnamed: 0.3', 'Unnamed: 0.4'], inplace=True)


In [38]:
df_final.columns.tolist()

['title',
 'url',
 'precio',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'detalle',
 'consumoce_ano',
 'letrace',
 'emisiones_co2',
 'tipologia',
 'propiedad_completa',
 'estado',
 'amueblado',
 'planta',
 'habitaciones',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'calefaccion',
 'ano_construccion',
 'plantas_edificio',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'disponibilidad',
 'nuda_propiedad',
 'sup_comercial',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 

In [51]:
df_final["descuento"].value_counts(dropna=False)

descuento
NaN       10367
-5,0%         4
-7,5%         3
-5,9%         3
-7,7%         3
-6,3%         2
-9,3%         2
-8,3%         2
-5,6%         2
-7,1%         2
-7,8%         2
-5,3%         2
-5,4%         2
-6,8%         2
-5,2%         2
-9,1%         1
-13,9%        1
-10,1%        1
-23,1%        1
-6,9%         1
-8,2%         1
-8,7%         1
-10,6%        1
-11,1%        1
-10,3%        1
-6,1%         1
-13,2%        1
-5,5%         1
-7,3%         1
-7,2%         1
-5,7%         1
-11,2%        1
-9,8%         1
-6,2%         1
-9,2%         1
-9,6%         1
Name: count, dtype: int64

In [54]:
# Eliminar el símbolo '%' y convertir a tipo numérico
df_final['descuento'] = df_final['descuento'].str.replace('%', '')
df_final['descuento'] = pd.to_numeric(df_final['descuento'].str.replace(',', '.'))

In [10]:
df_final["alarma"] = df_final["alarma"].fillna(df_final["sistema_alarma"])

uno puerta_seguridad y puerta_blindada en una columna

In [21]:
df_final["puerta_seguridad"] = df_final["puerta_seguridad"].fillna(df_final["puerta_blindada"])

In [29]:

def clasificar_consumo(valor):
    try:
        valor = float(valor)
        if valor < 10:
            return 'A'
        elif valor <= 20:
            return 'B'
        elif valor <= 35:
            return 'C'
        elif valor <= 55:
            return 'D'
        elif valor <= 75:
            return 'E'
        elif valor <= 100:
            return 'F'
        else:
            return 'G'
    except ValueError:
        return None



In [31]:
#compruebo si tengo valores en la columna consumoce_ano que no tengal en letrace, A: Menos de 10 kg CO2/m²/año,B: 10-20 kg CO2/m²/año,C: 20-35 kg CO2/m²/año,D: 35-55 kg CO2/m²/año
#E: 55-75 kg CO2/m²/año,F: 75-100 kg CO2/m²/año,G: Más de 100 kg CO2/m²/año
df_final.query("consumoce_ano.notnull() and letrace.isnull()")[["url","consumoce_ano"]]

,url,consumoce_ano


In [30]:
# Aplicar la función a la columna consumo_ce solo cuando letrace es nulo y consumo_ce no es nulo
df_final['letrace'] = df_final.apply(lambda row: clasificar_consumo(row['consumoce_ano']) if pd.isnull(row['letrace']) and not pd.isnull(row['consumoce_ano']) else row['letrace'], axis=1)

In [ ]:
# Aplicar la función a la columna consumo_ce
df['categoria'] = df['consumo_ce'].apply(clasificar_consumo)

# Mostrar el dataframe resultante
print(df)

In [13]:
df_final.to_csv("pisoscompletofinal.csv",index='False')

In [48]:

dfimetros = pd.read_excel("indomiosolometros.xlsx")

In [41]:
dfimetros.columns

Index(['url', 'm2_constr'], dtype='object')

In [ ]:
actualizar_valores(df_final, dfimetros, "url", "m2_constr")

In [119]:
# Actualizar las filas donde 'cantidad_armarios' no es nulo y 'tiene_armario' es 0 o nulo
df_final.loc[dfEDA['cantidad_armarios'].notnull() & (df_final['tiene_armario'].isnull() | df_final['tiene_armario'] == 0), 'tiene_armario'] = 1

In [5]:
df_final= df_final[df_final['tipologia'] != 'Negocio']

In [11]:
df_final.query("tipologia=='Venta'")[["url","web"]]

,url,web
7990,https://www.redpiso.es/inmueble/en-venta-en-calle-bravo-murillo-vallehermoso-chamberi-madrid-madrid-RP1812023119620,redpiso
8017,https://www.redpiso.es/inmueble/en-venta-en-calle-la-alegria-de-la-huerta-los-angeles-villaverde-madrid-madrid-RP332024131222,redpiso
9205,https://www.redpiso.es/inmueble/en-venta-en-plaza-pena-horcajo-penagrande-fuencarral-el-pardo-madrid-madrid-RP722023118896,redpiso


In [12]:
df_final['tipologia'] = df_final['tipologia'].replace('Venta', 'Piso')

In [130]:
# Actualizar los valores de 'distrito' y 'barrio' cuando 'cod_barrio' es 195
df_final.loc[df_final['cod_barrio'] == 195.00, ['distrito', 'barrio']] = ['Vicalvaro', 'Los Berrocales']

In [ ]:
df_final.query("cod_barrio.isnull()")[["url","distrito","cod_distrito","barrio","cod_barrio","descripcion_extendida"]]

In [140]:
df_final.loc[dfEDA['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3863841-piso-en-venta-en-madrid'), ['cod_distrito','distrito','cod_barrio','barrio']] =[13,'Puente de Vallecas',133,'Palomeras Bajas']
df_final.loc[dfEDA['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4086493-piso-en-venta-en-madrid'), ['cod_distrito','distrito','cod_barrio','barrio']] =[17,'Villaverde',171,'Villaverde Alto']

In [4]:

df_final = pd.read_csv("pisoscompletofinal.csv", index_col=False)

C:\Users\extas\AppData\Local\Temp\ipykernel_16784\1533870066.py:1: DtypeWarning: Columns (11,17,18,19,20,25,26,27,29,30,34,35,43,46,76,78,80,82,84,85,87,90,91,93,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv("pisoscompletofinal.csv", index_col=False)


# creacion dataframe EDA

In [7]:
df_final.columns

Index(['Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1',
       'Unnamed: 0', 'title', 'url', 'precio', 'precio_anterior', 'descuento',
       ...
       'hog_con_6', 'hog_con_7', 'hog_con_8', 'hog_con_9', 'hog_con_10',
       'hog_con_11', 'hog_con_12', 'hog_con_13', 'hog_con_14',
       'hog_con_15 y más'],
      dtype='object', length=145)

In [14]:
columnas_eliminar=["Unnamed: 0.4","Unnamed: 0.3", "Unnamed: 0.2","Unnamed: 0.1","Unnamed: 0",'title','precio_anterior','EUR/m2','zona','calle','detalle','emisiones_co2','propiedad_completa','habitaciones','balcon',"plantas_edificio",
"inmueble_ingresos","alquiler_opcion_a_compra","disponibilidad","nuda_propiedad",
'sup_comercial',"tipo_inmueble",'fibra_optica','instalacion_tv_centralizada',
'instalacion_tv_individual',"lujosa",'descripcion_extendida',"web","fecha_descarga","piscina",
'referencia',"fecha_publicacion","titular_anuncio","num_plantas","num_pisos","reservado",
"nombre_oficina","cod_barrio_2","aire_acondicionado","calefaccion",'campo12','sistema_alarma','puerta_blindada','descuento','consumoce_ano','cantidad_armarios',"cocina"]



dfEDA = df_final.drop(columns=columnas_eliminar)

# Ruta relativa del archivo
ruta_guardado_relativa = '../../02-EDA-y-feature-engineering/data/pisosEDA.csv'

# Crear el directorio de destino si no existe
directorio_destino = os.path.dirname(ruta_guardado_relativa)
os.makedirs(directorio_destino, exist_ok=True)

# Guardar el DataFrame dfEDA en la ruta relativa especificada
dfEDA.to_csv(ruta_guardado_relativa, index=False)

In [79]:
dfEDA.shape

(10421, 102)

In [47]:
dfEDA.columns.tolist()

['url',
 'precio',
 'descuento',
 'm2_constr',
 'cod_distrito',
 'cod_barrio',
 'consumoce_ano',
 'letrace',
 'tipologia',
 'estado',
 'amueblado',
 'planta',
 'dormitorios',
 'banos',
 'garaje',
 'trastero',
 'ascensor',
 'porteria',
 'ano_construccion',
 'jardin',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 'chimenea',
 'cocina',
 'exterior',
 'interior',
 'interior_y_exterior',
 'tiene_piscina',
 'porton_electrico',
 'puerta_blindada',
 'alarma',
 'videoportero',
 'acceso_discapacidad',
 'certificado_energetico',
 'tiene_jardin',
 'cantidad_visitas',
 'campo12',
 'cantidad_armarios',
 'agua_caliente',
 'tipo_suelo',
 'm2_utiles',
 'gastos_comunidad',
 'zonas_verdes',
 'puerta_seg

In [32]:
dfEDA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10427 entries, 0 to 10426
Columns: 102 entries, url to hog_con_15 y más
dtypes: float64(72), object(30)
memory usage: 8.1+ MB
